# Parsing Kommersant

In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import random
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import requests
import os
import time
import random
from random_user_agent.user_agent import UserAgent
from random_user_agent.params import SoftwareName, OperatingSystem
from threading import Thread, Lock
import threading
import pickle
import datetime

# import sys
# module_path = os.path.abspath(os.path.join('..'))
# if module_path not in sys.path:
#     sys.path.append(module_path+"\\src")

In [7]:
import os
import numpy as np

In [4]:
print(os.getcwd())

D:\Int dev\Diplom\ML model


In [ ]:
software_names = [SoftwareName.CHROME.value]
operating_systems = [OperatingSystem.WINDOWS.value, OperatingSystem.LINUX.value]   

user_agent_rotator = UserAgent(software_names=software_names, operating_systems=operating_systems, limit=1000)

In [15]:
#По номерам рубрик скачиваем ссылки
from parsing import load_links
#for rub in ['4', '40', '41']:
#    load_links(rub)

In [8]:
rubrics = {
    '2': 'politics', '3': 'economics', 
    '4': 'business', '40':'finance', '41': 'consumer_market'}
for rub in rubrics.values():
    urls = []
    lst = os.listdir(f'news_links/komersant_{rub}')
    lst = [el for el in lst if el[0] != '.']
    for file in lst:
        with open(f'news_links/komersant_{rub}/{file}') as file:
            urls_file = file.read()
            urls_file = urls_file.split('\n')
            urls.extend(urls_file)

    urls = np.unique(urls).tolist()

    with open(f'news_links/komersant_{rub}/urls.txt', 'w') as f:
        for url in urls:
            f.write(f'{url}\n')

# Parsing news texts 

In [1]:
with open(f'news_links/komersant_economics/urls.txt') as file:
    urls = file.read()
    urls = urls.split('\n')

In [2]:
from parsing import start_parsing, parse, parse_from_link_komersant

In [3]:
# Создаем массив данных, куда будут все склвдывать потоки
# data = {'link': [], 'data_or_ex': []}
# current_val = 0
total_news = len(urls)
print(total_news)
start_parsing('news_links/komersant_economics/raw_cites', parse, parse_from_link_komersant, urls, total_news, n_threads=7, save_evry=200)

25090

Invalid URL '': No scheme supplied. Perhaps you meant http://?
0 saved
200 saved
400 saved
600 saved
800 saved
1000 saved
1200 saved
1400 saved
1600 saved
1800 saved
2000 saved
2200 saved
2400 saved
2600 saved
2800 saved
3000 saved
3200 saved
3400 saved
3600 saved
3800 saved
4000 saved
4200 saved
4400 saved
4600 saved
4800 saved
5000 saved
5200 saved
5400 saved
5600 saved
5800 saved
6000 saved
6200 saved
6400 saved
6600 saved
6800 saved
7000 saved
7200 saved
7400 saved
7600 saved
7800 saved
8000 saved
8200 saved
8400 saved
8600 saved
8800 saved
9000 saved
9200 saved
9400 saved
9600 saved
9800 saved
10000 saved
10200 saved
10400 saved
10600 saved
10800 saved
11000 saved
11200 saved
11400 saved
11600 saved
11800 saved
12000 saved
12200 saved
12400 saved
12600 saved
12800 saved
13000 saved
13200 saved
13400 saved
13600 saved
13800 saved
14000 saved
14200 saved
14400 saved
14600 saved
14800 saved
15000 saved
15200 saved
15400 saved
15600 saved
15800 saved
16000 saved
16200 saved
164

In [5]:
import pickle
with open('news_links/komersant_business/raw_cites/200.pkl', 'rb') as f:
    data = pickle.load(f)
    print(data)

{'link': ['https://www.kommersant.ru//doc/3185981', 'https://www.kommersant.ru//doc/3186033', 'https://www.kommersant.ru//doc/3186054', 'https://www.kommersant.ru//doc/3186092', 'https://www.kommersant.ru//doc/3186215', 'https://www.kommersant.ru//doc/3186261', 'https://www.kommersant.ru//doc/3186294', 'https://www.kommersant.ru//doc/3186305', 'https://www.kommersant.ru//doc/3186329', 'https://www.kommersant.ru//doc/3186346', 'https://www.kommersant.ru//doc/3186349', 'https://www.kommersant.ru//doc/3186356', 'https://www.kommersant.ru//doc/3186382', 'https://www.kommersant.ru//doc/3186433', 'https://www.kommersant.ru//doc/3186439', 'https://www.kommersant.ru//doc/3186456', 'https://www.kommersant.ru//doc/3186468', 'https://www.kommersant.ru//doc/3186477', 'https://www.kommersant.ru//doc/3186485', 'https://www.kommersant.ru//doc/3186503', 'https://www.kommersant.ru//doc/3186519', 'https://www.kommersant.ru//doc/3186520', 'https://www.kommersant.ru//doc/3186528', 'https://www.kommersant.

In [4]:
import os
os.cpu_count()

4

In [5]:
pip install pymystem3

Note: you may need to restart the kernel to use updated packages.


# Preprocess text

In [4]:
# import re
import os
# import warnings

# import numpy as np
import pandas as pd
from pymystem3 import Mystem
import nltk

In [5]:
mystem = Mystem()
nltk.download('stopwords')
swords = set(stopwords.words('russian'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zolgu\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [9]:
from text_preprocess import lemm_file
root = 'news_links/komersant_finance'
files = sorted(os.listdir(f'{root}/raw_cites'))
files = [f for f in files if f[0] != '.']
c = 1

for file in files:
    print(f'file {file} in work')
    df = lemm_file(root, f'/{file}', mystem, swords)
    if df is None:
        continue
    df.to_csv(f'{root}/prepared_cites/{file[:-4]}.csv', index=False)
    print(f'{c} из {len(files)} готов')
    c += 1

file 1000.pkl in work
title ready
announce ready
text ready
1 из 127 готов
file 10000.pkl in work
title ready
announce ready
text ready
2 из 127 готов
file 10200.pkl in work
title ready
announce ready
text ready
3 из 127 готов
file 10400.pkl in work
title ready
announce ready
text ready
4 из 127 готов
file 10600.pkl in work
title ready
announce ready
text ready
5 из 127 готов
file 10800.pkl in work
title ready
announce ready
text ready
6 из 127 готов
file 11000.pkl in work
title ready
announce ready
text ready
7 из 127 готов
file 11200.pkl in work
title ready
announce ready
text ready
8 из 127 готов
file 11400.pkl in work
title ready
announce ready
text ready
9 из 127 готов
file 11600.pkl in work
title ready
announce ready
text ready
10 из 127 готов
file 11800.pkl in work
title ready
announce ready
text ready
11 из 127 готов
file 1200.pkl in work
title ready
announce ready
text ready
12 из 127 готов
file 12000.pkl in work
title ready
announce ready
text ready
13 из 127 готов
file 12200

title ready
announce ready
text ready
108 из 127 готов
file 6400.pkl in work
title ready
announce ready
text ready
109 из 127 готов
file 6600.pkl in work
title ready
announce ready
text ready
110 из 127 готов
file 6800.pkl in work
title ready
announce ready
text ready
111 из 127 готов
file 7000.pkl in work
title ready
announce ready
text ready
112 из 127 готов
file 7200.pkl in work
title ready
announce ready
text ready
113 из 127 готов
file 7400.pkl in work
title ready
announce ready
text ready
114 из 127 готов
file 7600.pkl in work
title ready
announce ready
text ready
115 из 127 готов
file 7800.pkl in work
title ready
announce ready
text ready
116 из 127 готов
file 800.pkl in work
title ready
announce ready
text ready
117 из 127 готов
file 8000.pkl in work
title ready
announce ready
text ready
118 из 127 готов
file 8200.pkl in work
title ready
announce ready
text ready
119 из 127 готов
file 8400.pkl in work
title ready
announce ready
text ready
120 из 127 готов
file 8600.pkl in work


In [1]:
from text_preprocess import komersant_to_standard_date
folders = ['news_links/komersant_' + t for t in ['economics', 'business', 'finance']]

komersant_to_standard_date(folders)

Обработано 100 из 126
Обработано 100 из 208
Обработано 200 из 208
Обработано 100 из 127


In [1]:
companies = {
    'ABRD': 'Абрау-Дюрсо ПАО ао', 
    'AFLT': 'Аэрофлот-росс.авиалин(ПАО)ао', 
    'ALRS': 'АЛРОСА ПАО ао',
    'APTK': 'ПАО "Аптечная сеть 36,6" ао',
    'CHMF': 'Северсталь (ПАО)ао',
    'GAZP': '"Газпром" (ПАО) ао', 
    'GMKN': 'ГМК "Нор.Никель" ПАО ао', 
    'LKOH': 'НК ЛУКОЙЛ (ПАО) - ао',
    'LSRG': 'Группа ЛСР ПАО ао',
    'MGNT': '"Магнит" ПАО ао',
    'MGTS': '"ПАО "МГТС" ао',
    'MOEX': 'ПАО Московская Биржа', 
    'MVID': '"М.видео" ПАО ао',
    'NAUK': 'НПО Наука ао',
    'PIKK': 'ПИК СЗ (ПАО) ао', 
    'PLZL': 'Полюс ПАО ао',
    'RBCM': 'ГК РБК ПАО ао',
    'ROSN': 'ПАО НК Роснефть',
    'RTKM': 'Ростелеком (ПАО) ао.',  
    'SBER': 'Сбербанк России ПАО ао',
    'TATN': 'ПАО "Татнефть" ао',
    'VTBR': 'ао ПАО Банк ВТБ'
    }

In [2]:
for_regex = {
    'ABRD': ['абрау-дюрсо', 'абрау', 'abrd'],
    'ALRS': ['алроса', 'alrs'], 
    'AFLT': ['аэрофлот', 'aflt'],
    'APTK': ['36.6', '36,6', 'aptk'],
    'CHMF': ['cеверсталь', 'chmf'],
    'VTBR': ['втб', 'vtbr'], 
    'GAZP': ['газпром', 'gazp'], 
    'GMKN': ['норникель', 'нор никель', 'норильский никель', 'gmkn'],
    'LKOH': ['лукойл', 'lkoh'],
    'LSRG': ['лср', 'lsr', 'lsrg'], 
    'MGNT': ['магнит', 'magnit', 'mgnt'],
    'MGTS': ['мгтс', 'московская городская телефонная сеть', 'mgts'],
    'MOEX': ['московский биржа', 'мосбиржа', 'мос биржа', 'мос. биржа', 'moex'],
    'MVID': ['мвидео', 'м. видео', 'м.видео', 'mvideo', 'm. video', 'm.video', 'mvid'],
    'NAUK': ['нпо наука', 'nauk'],
    'PIKK': ['pik', 'пик', 'первая ипотечная компания', 'pikk'], 
    'PLZL': ['plzl', 'полюс золото'],
    'RBCM': ['rbc', 'рбк', 'росбизнесконсалтинг', 'ragr'],
    'ROSN': ['роснефть', 'рос нефть', 'rosneft', 'rosn'],
    'RTKM': ['ростелеком', 'рос телеком', 'rtkm'],  
    'SBER': ['/^сбер$/', 'сбербанк', 'sber'],   
    'TATN': ['tatn', 'tatneft', 'татнефть'], 
    'VTBR': ['втб', 'vtb', 'vtbr']
}

In [ ]:
## Удаленные котировки: RAGR, SMLT, LENT

In [3]:
folders = ['komersant_business', 'komersant_economics', 'komersant_finance']
roots = ['news_links/' + el for el in folders]

In [4]:
from text_preprocess import add_company
add_company(roots, for_regex, companies)

news_links/komersant_business done
news_links/komersant_economics done
news_links/komersant_finance done


In [5]:
import pandas as pd

In [9]:
for root in roots:
    df = pd.read_csv(f'news_links/komersant_business/prepared_cites/200.csv')
    for company in companies:
        reg = for_regex[company]
        reg = ' | '.join(reg)
        in_str = df['title'].str.contains(reg) | df['announce'].str.contains(reg) | df['text'].str.contains(reg)
        df[company] = in_str
    print(df)

                                       link                 date  \
0    https://www.kommersant.ru//doc/3185981  2017-01-01 10:50:40   
1    https://www.kommersant.ru//doc/3186033  2017-01-02 03:46:53   
2    https://www.kommersant.ru//doc/3186054  2017-01-02 09:57:50   
3    https://www.kommersant.ru//doc/3186092  2017-01-02 19:15:34   
4    https://www.kommersant.ru//doc/3186215  2017-01-03 04:36:36   
..                                      ...                  ...   
195  https://www.kommersant.ru//doc/3195398  2017-01-18 15:38:31   
196  https://www.kommersant.ru//doc/3195421  2017-01-18 16:15:34   
197  https://www.kommersant.ru//doc/3195451  2017-01-18 16:52:46   
198  https://www.kommersant.ru//doc/3195465  2017-01-18 17:15:13   
199  https://www.kommersant.ru//doc/3195473  2017-01-18 17:26:13   

                                                 title     announce  \
0            россия 1 январь повышаться акциз алкоголь  no announce   
1    прямой авиасообщение открываться соч

# Сбор котировок

In [32]:
import requests
import csv
import xml.etree.ElementTree as ET
import pandas as pd
import time
import random

In [40]:
d = {'close': [], 'date': []}

for company in companies:
    data = pd.DataFrame.from_dict(d)
    index = 0
    total = 100
    while index < total:
        link = f'https://iss.moex.com/iss/history/engines/stock/markets/shares/boards/TQBR/securities/{company}.xml?iss.meta=off&from=2017-01-01&till=2025-04-01&history.columns=LEGALCLOSEPRICE,TRADEDATE&start={index}'
        download = requests.get(link)
        decoded_content = download.content.decode('utf-8')
        root = ET.fromstring(decoded_content)
        index = int(root[1][0][0].get('INDEX'))
        index += 100
        total = int(root[1][0][0].get('TOTAL'))
        rows = root[0][0]
        for row in rows:
            close = row.get('LEGALCLOSEPRICE')
            date = row.get('TRADEDATE')
            data.loc[len(data)] = [close, date]
        time.sleep(1 + random.random())
    print(company + ' with ' + str(total) + 'quotes finished')
    data.to_csv(f'quotes_moex/{company}.csv', index=False)
    

ABRD with 2087quotes finished
AFLT with 2087quotes finished
ALRS with 2087quotes finished
APTK with 2087quotes finished
CHMF with 2087quotes finished
GAZP with 2087quotes finished
GMKN with 2087quotes finished
LENT with 846quotes finished
LKOH with 2087quotes finished
LSRG with 2087quotes finished
MGNT with 2087quotes finished
MGTS with 2087quotes finished
MOEX with 2087quotes finished
MVID with 2087quotes finished
NAUK with 2087quotes finished
PIKK with 2087quotes finished
PLZL with 2087quotes finished
RAGR with 32quotes finished
RBCM with 1992quotes finished
ROSN with 2087quotes finished
RTKM with 2087quotes finished
SBER with 2087quotes finished
SMLT with 1123quotes finished
TATN with 2087quotes finished
VTBR with 2087quotes finished


In [22]:
link = 'https://iss.moex.com/iss/history/engines/stock/markets/shares/boards/TQBR/securities/AFLT.xml?iss.meta=off&from=2017-01-01&till=2025-04-01&history.columns=TRADEDATE,SECID,LEGALCLOSEPRICE'
download = requests.get(link)
decoded_content = download.content.decode('utf-8')
root = ET.fromstring(decoded_content)

print(root[1][0][0].attrib)
print(root[1][0][0].get('INDEX'))
print(root[1][0][0].get('TOTAL'))

{'INDEX': '0', 'TOTAL': '2087', 'PAGESIZE': '100'}
0
2087


In [27]:
d = {'close': [], 'date': []}
data = pd.DataFrame.from_dict(d)
print(data)

Empty DataFrame
Columns: [close, date]
Index: []


# Тест данных с Финам и не только

In [8]:
#Собираем данные дат в один файл
import pandas as pd

data = pd.DataFrame()
    
try:
    tmp = pd.read_csv(f'D:/Загрузки с яндекса/AFLT_250416_250421.csv', sep=';')[['<DATE>', '<TIME>', '<CLOSE>']]
    time = tmp['<DATE>'].astype(str)
    tmp['timestamp'] = pd.to_datetime(time, format='%y%m%d')
    tmp.sort_values('timestamp', inplace=True)
       
    tmp.drop(columns=['<DATE>', '<TIME>'], inplace=True)
    tmp.columns = ['close', 'timestamp']
    tmp.sort_values('timestamp', inplace=True)
    data = pd.concat([data, tmp], axis=0)
except Exception as ex:
    print(ex)
print(data)

   close  timestamp
0  69.35 2025-04-16
1  71.15 2025-04-17
2  70.18 2025-04-18
3  72.45 2025-04-21
4  72.53 2025-04-22


In [5]:
import pandas as pd

df = pd.read_csv(f'news_links/komersant_economics/prepared_cites/200.csv')
df = df[df['date'] != 'No time']
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
# df['date'] = pd.to_datetime(df['date'])
print(df['date'])

ValueError: time data '2017-01-02 21:23:59' does not match format '%Y%m%d' (match)

# Добавляем таргет и дату

In [8]:
from text_preprocess import add_target
# folders = ['news_links/komersant_' + t for t in ['economics','business', 'finance']]
folders = ['news_links/komersant_' + t for t in ['economics']]

for folder in folders:
    add_target(folder, companies)
    print(f'{folder} готов')

file gathered
ABRD готов
AFLT готов
ALRS готов
APTK готов
CHMF готов
GAZP готов
GMKN готов
LKOH готов
LSRG готов
MGNT готов
MGTS готов
MOEX готов
MVID готов
NAUK готов
PIKK готов
PLZL готов
RBCM готов
ROSN готов
RTKM готов
SBER готов
TATN готов
VTBR готов
news_links/komersant_economics готов
file gathered
ABRD готов
AFLT готов
ALRS готов
APTK готов
CHMF готов
GAZP готов
GMKN готов
LKOH готов
LSRG готов
MGNT готов
MGTS готов
MOEX готов
MVID готов
NAUK готов
PIKK готов
PLZL готов
RBCM готов
ROSN готов
RTKM готов
SBER готов
TATN готов
VTBR готов
news_links/komersant_business готов
file gathered
ABRD готов
AFLT готов
ALRS готов
APTK готов
CHMF готов
GAZP готов
GMKN готов
LKOH готов
LSRG готов
MGNT готов
MGTS готов
MOEX готов
MVID готов
NAUK готов
PIKK готов
PLZL готов
RBCM готов
ROSN готов
RTKM готов
SBER готов
TATN готов
VTBR готов
news_links/komersant_finance готов


In [14]:
import pandas as pd
tmp = pd.read_csv(f'news_links/komersant_economics/targeted.csv', sep=',')
print(tmp.isna().sum())

link                  0
date                  0
title                 5
announce              0
text                 21
1 день ABRD close     0
1 день AFLT close     0
1 день ALRS close     0
1 день APTK close     0
1 день CHMF close     0
1 день GAZP close     0
1 день GMKN close     0
1 день LKOH close     0
1 день LSRG close     0
1 день MGNT close     0
1 день MGTS close     0
1 день MOEX close     0
1 день MVID close     0
1 день NAUK close     0
1 день PIKK close     0
1 день PLZL close     0
1 день RBCM close     0
1 день ROSN close     0
1 день RTKM close     0
1 день SBER close     0
1 день TATN close     0
1 день VTBR close     0
dtype: int64


In [19]:
folders = ['news_links/komersant_' + t for t in ['economics','business', 'finance']]
# folders = ['news_links/komersant_' + t for t in ['economics']]

for folder in folders:
    print(folder)
    df = pd.read_csv(f'{folder}/targeted.csv')
    print(len(df))
#     print(df.isna().sum())
    df = df.dropna()
    print(len(df))
#     print(df.isna().sum())
    df.to_csv(f'{folder}/targeted.csv', index=False)


news_links/komersant_economics
25061
25061
news_links/komersant_business
41514
41465
news_links/komersant_finance
25347
25093


In [27]:
folders = ['news_links/komersant_' + t for t in ['business', 'finance']]
# folders = ['news_links/komersant_' + t for t in ['economics']]

for folder in folders:
    print(folder)
    df = pd.read_csv(f'{folder}/targeted.csv')
    print(len(df))
    print(len(df[df['announce'] != 'no announce']))
    print(len(df[df['title'] != 'no title']))
    print(len(df[df['text'] != 'no text']))
    df = df.drop('announce', axis=1)
    df.to_csv(f'{folder}/targeted.csv', index=False)
    

news_links/komersant_business
41465
10750
41465
41465
news_links/komersant_finance
25093
10052
25093
25093


# Добавляем принадлежность к компании и отрасли

In [1]:
folders = ['news_links/komersant_' + t for t in ['economics','business', 'finance']]

In [4]:
from text_preprocess import add_company
add_company(folders, for_regex, companies)

news_links/komersant_economics done
news_links/komersant_business done
news_links/komersant_finance done


In [7]:
import pandas as pd
# folders = ['news_links/komersant_' + t for t in ['business', 'finance']]
# folders = ['news_links/komersant_' + t for t in ['business']]

for folder in folders:
    print(folder)
    df = pd.read_csv(f'{folder}/targeted.csv')
    print(len(df))
    print(df.value_counts('GAZP'))
    

news_links/komersant_business
41465
GAZP
False    36685
True      4780
dtype: int64


In [7]:
companies = {
    'ABRD': 'Абрау-Дюрсо ПАО ао', 
    'AFLT': 'Аэрофлот-росс.авиалин(ПАО)ао', 
    'ALRS': 'АЛРОСА ПАО ао',
    'APTK': 'ПАО "Аптечная сеть 36,6" ао',
    'CHMF': 'Северсталь (ПАО)ао',
    'GAZP': '"Газпром" (ПАО) ао', 
    'GMKN': 'ГМК "Нор.Никель" ПАО ао', 
    'LKOH': 'НК ЛУКОЙЛ (ПАО) - ао',
    'LSRG': 'Группа ЛСР ПАО ао',
    'MGNT': '"Магнит" ПАО ао',
    'MGTS': '"ПАО "МГТС" ао',
    'MOEX': 'ПАО Московская Биржа', 
    'MVID': '"М.видео" ПАО ао',
    'NAUK': 'НПО Наука ао',
    'PIKK': 'ПИК СЗ (ПАО) ао', 
    'PLZL': 'Полюс ПАО ао',
    'RBCM': 'ГК РБК ПАО ао',
    'ROSN': 'ПАО НК Роснефть',
    'RTKM': 'Ростелеком (ПАО) ао.',  
    'SBER': 'Сбербанк России ПАО ао',
    'TATN': 'ПАО "Татнефть" ао',
    'VTBR': 'ао ПАО Банк ВТБ'
    }

In [2]:
for_regex_industry = {
    'finance': ['ипотека', 'sber', '/^сбер$/', 'сбербанк', 'банк', 'кредит', 'депозит', 'мкб', 'московский кредитный банк', 'cbom', 'vtbr', 'втб', 'финансовый', 'финансы', 'биржа', 'moex', 'tcsg', 'тинькофф', 'ценные бумаги', 'ценная бумага', 'облигация', 'облигации', 'паевый', 'фонд', 'котировка'],
    'gold': ['poly', 'plzl', 'полиметалл', 'полюс золото', 'золото', 'pogr', 'petropavlovsk plc', 'petropavlovsk'],
    'ferrous_metallurgy': ['железо', 'сталь', 'северсталь', 'чугун', 'ммк', 'магнитогорский металлургический комбинат', 'magn', 'северсталь', 'chmf', 'nlmk', 'новолипецкий металлургический комбинат', 'металл'],
    'oil_gas':['gazp', 'газпром', 'rosn', 'роснефть', 'сургутнефтегаз', 'sngs', 'газ', 'нефть', 'нефтепровод', 'газопровод', 'татнефть', 'tatn', 'лукойл', 'lkoh', 'транснефть', 'trnfp', 'новатэк', 'nvtk'],
    'non_ferrous_metallurgy': ['rual', 'rusal', 'uc rusal','русал', 'gmkn', 'норникель', 'норильский никель', 'ниель', 'алюминий', 'палладий', 'олово', 'медь', 'глинозем'],
    'electrical networks': ['интер рао', 'irao', 'русгидро', 'hydr', 'элетричество', 'гэс', 'тэц', 'энергетика', 'фск еэс', 'fees'],
    'telecom': ['мтс', 'mtss', 'ростелеком', 'rtkm', 'сеть', 'связь', 'интернет', 'провайдер'],
    'it': ['машинный обучение', 'цифровой технология', 'облако', 'вк', 'мэйл', 'вконтакте', 'mail', 'vk', 'vkco', 'яндекс', 'yandex', 'yndx'],
    'real_estate': ['недвижимость', 'новостройка', 'ипотека', 'лср', 'lsrg', 'пик', 'pikk'],
    'consume_retail': ['напиток', 'вода', 'абрау', 'дюрсо', 'abrd', 'алкоголь', 'вино', 'еда', 'пищевой', 'пища', 'магнит', 'перекресток', 'пятерочка', 'продукт', 'магазин'],
    'tech_market': ['м.видео', 'mvideo', 'm. video', 'dns', 'citilink', 'смартфон', 'ноутбук', 'телевизор', 'бытовой', 'техника'],
    'research': ['исследование', 'наука', 'научный', 'nauk', 'нпо'],
}

industries = for_regex_industry.keys()

In [3]:
from text_preprocess import add_industry
add_industry(folders, industries, for_regex_industry)

news_links/komersant_economics done
news_links/komersant_business done
news_links/komersant_finance done


In [9]:
import pandas as pd
# folders = ['news_links/komersant_' + t for t in ['business', 'finance']]
folders = ['news_links/komersant_' + t for t in ['business']]

for folder in folders:
    print(folder)
    df = pd.read_csv(f'{folder}/targeted.csv')
    print(len(df))
    print(df.value_counts('finance'))

news_links/komersant_business
41465
finance
False    27209
True     14256
dtype: int64


In [11]:
folders = ['news_links/komersant_' + t for t in ['economics','business', 'finance']]
end_df = pd.DataFrame()
for folder in folders:
    print(folder)
    df = pd.read_csv(f'{folder}/targeted.csv')
    end_df = pd.concat([end_df, df])
    
print(len(end_df))
end_df.to_csv(f'news_links/final.csv', index=False)

news_links/komersant_economics
news_links/komersant_business
news_links/komersant_finance
91619


# Учим модель!

In [20]:
pip install lightgbm

     ---------------------------------------- 1.5/1.5 MB 5.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, cohen_kappa_score

In [10]:
import os
import pandas as pd
import numpy as np
import sys

In [4]:
news = pd.read_csv(f'news_links/final.csv')
news.drop(columns=['link', 'text'], inplace=True)

In [5]:
companies = {
    'ABRD': 'Абрау-Дюрсо ПАО ао', 
    'AFLT': 'Аэрофлот-росс.авиалин(ПАО)ао', 
    'ALRS': 'АЛРОСА ПАО ао',
    'APTK': 'ПАО "Аптечная сеть 36,6" ао',
    'CHMF': 'Северсталь (ПАО)ао',
    'GAZP': '"Газпром" (ПАО) ао', 
    'GMKN': 'ГМК "Нор.Никель" ПАО ао', 
    'LKOH': 'НК ЛУКОЙЛ (ПАО) - ао',
    'LSRG': 'Группа ЛСР ПАО ао',
    'MGNT': '"Магнит" ПАО ао',
    'MGTS': '"ПАО "МГТС" ао',
    'MOEX': 'ПАО Московская Биржа', 
    'MVID': '"М.видео" ПАО ао',
    'NAUK': 'НПО Наука ао',
    'PIKK': 'ПИК СЗ (ПАО) ао', 
    'PLZL': 'Полюс ПАО ао',
    'RBCM': 'ГК РБК ПАО ао',
    'ROSN': 'ПАО НК Роснефть',
    'RTKM': 'Ростелеком (ПАО) ао.',  
    'SBER': 'Сбербанк России ПАО ао',
    'TATN': 'ПАО "Татнефть" ао',
    'VTBR': 'ао ПАО Банк ВТБ'
    }

In [33]:
print(news.columns)
targets = news.iloc[:,2:24].columns
print(targets)
# sys.exit()
companies = {'AFLT': 'Аэрофлот-росс.авиалин(ПАО)ао'}

for company in companies:
    print(company)
    os.makedirs(f'news_links/companies_results/{company}', exist_ok=True)
    comp = news[news[company] == True].copy()
    comp.dropna(inplace=True)
    print(len(comp))
    print(comp.head(5))
#     break

    from sklearn.feature_extraction.text import TfidfVectorizer
    tfidf = TfidfVectorizer(sublinear_tf=True, min_df=0.1, ngram_range=(1,2))

    train, test = comp.iloc[:int(0.8*len(comp)), :], comp.iloc[int(0.8*len(comp)):, :]
    tfidf.fit(comp['title'])

    features_train = tfidf.transform(train['title'])
    features_test = tfidf.transform(test['title'])
    print(features_test[:10])
    print(features_test.shape)
    print(tfidf.get_feature_names_out())
    print(features_test.toarray())

    X_train = np.asarray(features_train.todense())
    X_test = np.asarray(features_test.todense())
#     print(X_test[:10])
#     print(len(X_test))
    
#     break

    tr = [col for col in targets if company in col]
    print(tr)

    test_logg = []
    train_logg = []

    for t in tr:
        y_train = train[t].values.astype(int)
#         print(y_train[:10])
#         print(len(y_train))
#         sys.exit()
        y_test = test[t].values.astype(int)
        print(y_test[:10])
        print(len(y_test))

        param_grid = {'max_depth' : list(range(3,13))}

        clf = RandomForestClassifier(random_state=123, criterion='entropy', n_jobs=-1)
        gc = GridSearchCV(estimator=clf, param_grid=param_grid, cv=2, n_jobs=1, scoring='accuracy', verbose=2)
        gc.fit(X_train, y_train)

        preds_test = gc.predict(X_test)
        print(preds_test)
        print(len(preds_test))
#         preds_train = gc.predict(X_train)

        preds_proba_test = gc.predict_proba(X_test)
        print(preds_proba_test)
        print(len(preds_proba_test))
#         preds_proba_train = gc.predict_proba(X_train)

#         test_logg.append(np.mean(y_test==preds_test))
#         train_logg.append(np.mean(y_train==preds_train))
        print(np.mean(y_test==preds_test))

#     with open(f'news_links/companies_results/{company}/{company}_train_logg.txt', 'w') as fp:
#         for item in train_logg:
#             fp.write("%s\n" % item)
#     with open(f'news_links/companies_results/{company}/{company}_test_logg.txt', 'w') as fp:
#         for item in test_logg:
#             fp.write("%s\n" % item)

  
    print(f'{company} done')

Index(['date', 'title', '1 день ABRD close', '1 день AFLT close',
       '1 день ALRS close', '1 день APTK close', '1 день CHMF close',
       '1 день GAZP close', '1 день GMKN close', '1 день LKOH close',
       '1 день LSRG close', '1 день MGNT close', '1 день MGTS close',
       '1 день MOEX close', '1 день MVID close', '1 день NAUK close',
       '1 день PIKK close', '1 день PLZL close', '1 день RBCM close',
       '1 день ROSN close', '1 день RTKM close', '1 день SBER close',
       '1 день TATN close', '1 день VTBR close', 'ABRD', 'AFLT', 'ALRS',
       'APTK', 'CHMF', 'GAZP', 'GMKN', 'LKOH', 'LSRG', 'MGNT', 'MGTS', 'MOEX',
       'MVID', 'NAUK', 'PIKK', 'PLZL', 'RBCM', 'ROSN', 'RTKM', 'SBER', 'TATN',
       'VTBR', 'finance', 'gold', 'ferrous_metallurgy', 'oil_gas',
       'non_ferrous_metallurgy', 'electrical networks', 'telecom', 'it',
       'real_estate', 'consume_retail', 'tech_market', 'research'],
      dtype='object')
Index(['1 день ABRD close', '1 день AFLT close', '1 д

D:\PROgrams_for_teach\anaconda\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(


[CV] END ........................................max_depth=3; total time=   0.1s
[CV] END ........................................max_depth=3; total time=   0.0s
[CV] END ........................................max_depth=4; total time=   0.0s
[CV] END ........................................max_depth=4; total time=   0.0s
[CV] END ........................................max_depth=5; total time=   0.0s
[CV] END ........................................max_depth=5; total time=   0.0s
[CV] END ........................................max_depth=6; total time=   0.0s
[CV] END ........................................max_depth=6; total time=   0.0s
[CV] END ........................................max_depth=7; total time=   0.1s
[CV] END ........................................max_depth=7; total time=   0.1s
[CV] END ........................................max_depth=8; total time=   0.0s
[CV] END ........................................max_depth=8; total time=   0.0s
[CV] END ...................

# Начинаем заново с добавления таргета - 2 класса

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
# from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, cohen_kappa_score
import os
import pandas as pd
import numpy as np
import sys

In [5]:
companies = {
    'ABRD': 'Абрау-Дюрсо ПАО ао', 
    'AFLT': 'Аэрофлот-росс.авиалин(ПАО)ао', 
    'ALRS': 'АЛРОСА ПАО ао',
    'APTK': 'ПАО "Аптечная сеть 36,6" ао',
    'CHMF': 'Северсталь (ПАО)ао',
    'GAZP': '"Газпром" (ПАО) ао', 
    'GMKN': 'ГМК "Нор.Никель" ПАО ао', 
    'LKOH': 'НК ЛУКОЙЛ (ПАО) - ао',
    'LSRG': 'Группа ЛСР ПАО ао',
    'MGNT': '"Магнит" ПАО ао',
    'MGTS': '"ПАО "МГТС" ао',
    'MOEX': 'ПАО Московская Биржа', 
    'MVID': '"М.видео" ПАО ао',
    'NAUK': 'НПО Наука ао',
    'PIKK': 'ПИК СЗ (ПАО) ао', 
    'PLZL': 'Полюс ПАО ао',
    'RBCM': 'ГК РБК ПАО ао',
    'ROSN': 'ПАО НК Роснефть',
    'RTKM': 'Ростелеком (ПАО) ао.',  
    'SBER': 'Сбербанк России ПАО ао',
    'TATN': 'ПАО "Татнефть" ао',
    'VTBR': 'ао ПАО Банк ВТБ'
    }

In [34]:
# companies = {
#     'ABRD': 'Абрау-Дюрсо ПАО ао', 
#     'AFLT': 'Аэрофлот-росс.авиалин(ПАО)ао' 
#     }

In [50]:
cols = ['1 день']
times = [pd.Timedelta(1, unit='day')]
deltas = {i:j for i, j in zip(cols, times)}
comp = [com for com in companies]
file = pd.read_csv(f'news_links/final.csv')

# file = file.head(10)[1:]

file['date'] = pd.to_datetime(file['date'])
file.sort_values('date', ignore_index=True, inplace=True)
for com in comp:
    price = pd.read_csv(f'quotes_moex/{com}.csv')
    price['date'] = pd.to_datetime(price['date']).dt.date
    price['date'] = pd.to_datetime(price['date'])
    price.sort_values('date', ignore_index=True, inplace=True)
    for col in cols:
            file['date'] -= deltas[col]
            file = pd.merge_asof(file, price, on='date', direction='backward')
            file.rename(columns={'close': f'{col} {com} close prev'}, inplace=True)
            file['date'] += deltas[col]
    print(f'{com} готов')
file.to_csv(f'news_links/final_v2.csv', index=False)

ABRD готов
AFLT готов
ALRS готов
APTK готов
CHMF готов
GAZP готов
GMKN готов
LKOH готов
LSRG готов
MGNT готов
MGTS готов
MOEX готов
MVID готов
NAUK готов
PIKK готов
PLZL готов
RBCM готов
ROSN готов
RTKM готов
SBER готов
TATN готов
VTBR готов


In [35]:
news = pd.read_csv(f'news_links/final_v2.csv')
news = news[news.date > '2017-01-03']
# news['message'] = news['title'] + ' ' + news['text']
news['message'] = news['title']
news.drop(columns=['link', 'title', 'text'], inplace=True)

In [36]:
count = 0
for company in companies:
    count += len(news[news[company] == True])
    
print(count)

39015


In [37]:
# print(news.columns)
# sys.exit()
df = pd.DataFrame()
for company in companies:
    comp = news[news[company] == True].copy()
    comp['close'] = comp[f'1 день {company} close'] - comp[f'1 день {company} close prev']
    comp = comp.loc[:, ['date', 'message', 'close', 
        'ABRD', 'AFLT', 'ALRS', 'APTK', 'CHMF', 'GAZP',
        'GMKN', 'LKOH', 'LSRG', 'MGNT', 'MGTS', 'MOEX', 'MVID', 'NAUK', 'PIKK',
        'PLZL', 'RBCM', 'ROSN', 'RTKM', 'SBER', 'TATN', 'VTBR', 'finance',
        'gold', 'ferrous_metallurgy', 'oil_gas', 'non_ferrous_metallurgy',
        'electrical networks', 'telecom', 'it', 'real_estate', 'consume_retail',
        'tech_market', 'research']]
    df = pd.concat([df, comp], axis=0)
df.sort_values('date', ignore_index=True, inplace=True)
print(df)

             date                                            message    close  \
0      2017-01-04  банк intesa обеспечивать половина сделка прива...  -20.500   
1      2017-01-04  роснефтегаз подтверждать закрытие сделка прода...  -20.500   
2      2017-01-04  иран допускать газпром лукойл участие тендер н... -196.000   
3      2017-01-04  иран допускать газпром лукойл участие тендер н...   -3.700   
4      2017-01-05  нафтогаз называть способ сохранение украина тр...   -3.340   
...           ...                                                ...      ...   
39010  2025-03-31  убыток спб биржа рсба снижаться 2024 год 10 6 ...   -2.750   
39011  2025-03-31                                  мфо занимать банк   11.400   
39012  2025-03-31  правительство поддерживать ограничение полномо...   -0.005   
39013  2025-03-31                                    трамплин золото   -1.730   
39014  2025-03-31  газпром существенно увеличивать поставка слова...   -2.750   

        ABRD   AFLT   ALRS 

In [38]:
print(len(df))

39015


In [39]:
df.drop_duplicates(inplace=True)
print(len(df))

38282


In [40]:
# df.drop(columns=['date'], inplace=True)
df['class'] = df['close'] > 0
# print(df)
# print(len(df))

In [41]:
df = df.replace({True: 1, False: 0})
# print(df)

In [42]:
df.drop(columns=['close', 'date'], inplace=True)
print(df)

                                                 message  ABRD  AFLT  ALRS  \
0      банк intesa обеспечивать половина сделка прива...     0     0     0   
1      роснефтегаз подтверждать закрытие сделка прода...     0     0     0   
2      иран допускать газпром лукойл участие тендер н...     0     0     0   
3      иран допускать газпром лукойл участие тендер н...     0     0     0   
4      нафтогаз называть способ сохранение украина тр...     0     0     0   
...                                                  ...   ...   ...   ...   
39010  убыток спб биржа рсба снижаться 2024 год 10 6 ...     0     0     0   
39011                                  мфо занимать банк     0     0     0   
39012  правительство поддерживать ограничение полномо...     0     0     0   
39013                                    трамплин золото     0     0     0   
39014  газпром существенно увеличивать поставка слова...     0     0     0   

       APTK  CHMF  GAZP  GMKN  LKOH  LSRG  ...  oil_gas  \
0   

In [43]:
print(len(df[df['class'] == 1]))

18791


In [80]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=0.3, max_df=0.7, ngram_range=(1,2))

# train, test = df.iloc[:int(0.8*len(df)), :], df.iloc[int(0.8*len(df)):, :]
features = tfidf.fit_transform(df['message'])

# features = tfidf.transform(train['message'])
# features_test = tfidf.transform(test['message'])

# print(features_train)
# print(features_test)

# X_train = np.asarray(features_train.todense())
# X_test = np.asarray(features_test.todense())


# print(X_test[0])
# print(len(X_test))

In [81]:
# print(features_train)
# print(features_test)
print(tfidf.get_feature_names_out())
print(features.toarray())

# print(X_test[0])
# print(len(X_test))

['банк' 'весь' 'время' 'господин' 'данные' 'заявлять' 'компания' 'конец'
 'который' 'крупный' 'материал' 'млн' 'млрд' 'млрд руб' 'мочь' 'новый'
 'объем' 'отмечать' 'первый' 'получать' 'российский' 'россия' 'рост' 'руб'
 'рынок' 'свой' 'слово' 'сообщать' 'составлять' 'становиться' 'считать'
 'также' 'уровень' 'цена' 'число']
[[0.60100448 0.         0.         ... 0.         0.         0.        ]
 [0.         0.315523   0.         ... 0.         0.         0.        ]
 [0.         0.28415424 0.         ... 0.         0.         0.32647345]
 ...
 [0.         0.         0.         ... 0.         0.         0.34074965]
 [0.12095655 0.12190579 0.         ... 0.28640263 0.51331176 0.23714434]
 [0.         0.24308843 0.         ... 0.         0.27604226 0.        ]]


In [82]:
text_df = pd.DataFrame(data=features.toarray(), columns=tfidf.get_feature_names_out())
print(text_df)

           банк      весь     время  господин    данные  заявлять  компания  \
0      0.601004  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
1      0.000000  0.315523  0.000000  0.000000  0.000000  0.363113  0.244868   
2      0.000000  0.284154  0.000000  0.000000  0.000000  0.000000  0.373379   
3      0.000000  0.284154  0.000000  0.000000  0.000000  0.000000  0.373379   
4      0.000000  0.196424  0.000000  0.204815  0.204917  0.226051  0.258101   
...         ...       ...       ...       ...       ...       ...       ...   
38285  0.190141  0.000000  0.000000  0.000000  0.000000  0.000000  0.251806   
38286  0.340238  0.103830  0.199624  0.000000  0.000000  0.000000  0.136433   
38287  0.000000  0.000000  0.000000  0.000000  0.000000  0.341313  0.000000   
38288  0.120957  0.121906  0.000000  0.000000  0.215329  0.000000  0.000000   
38289  0.000000  0.243088  0.000000  0.000000  0.000000  0.000000  0.188654   

          конец   который   крупный  ...      свой 

In [44]:
index_df = df.reset_index(drop=True)
print(index_df)

                                                 message  ABRD  AFLT  ALRS  \
0      банк intesa обеспечивать половина сделка прива...     0     0     0   
1      роснефтегаз подтверждать закрытие сделка прода...     0     0     0   
2      иран допускать газпром лукойл участие тендер н...     0     0     0   
3      иран допускать газпром лукойл участие тендер н...     0     0     0   
4      нафтогаз называть способ сохранение украина тр...     0     0     0   
...                                                  ...   ...   ...   ...   
38277  убыток спб биржа рсба снижаться 2024 год 10 6 ...     0     0     0   
38278                                  мфо занимать банк     0     0     0   
38279  правительство поддерживать ограничение полномо...     0     0     0   
38280                                    трамплин золото     0     0     0   
38281  газпром существенно увеличивать поставка слова...     0     0     0   

       APTK  CHMF  GAZP  GMKN  LKOH  LSRG  ...  oil_gas  \
0   

In [84]:
print(len(index_df))
print(len(text_df))
result_df = pd.concat([text_df, index_df], axis=1)
result_df.drop('message', axis=1, inplace=True)
print(result_df)
print(result_df.isna().sum())

38290
38290
           банк      весь     время  господин    данные  заявлять  компания  \
0      0.601004  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
1      0.000000  0.315523  0.000000  0.000000  0.000000  0.363113  0.244868   
2      0.000000  0.284154  0.000000  0.000000  0.000000  0.000000  0.373379   
3      0.000000  0.284154  0.000000  0.000000  0.000000  0.000000  0.373379   
4      0.000000  0.196424  0.000000  0.204815  0.204917  0.226051  0.258101   
...         ...       ...       ...       ...       ...       ...       ...   
38285  0.190141  0.000000  0.000000  0.000000  0.000000  0.000000  0.251806   
38286  0.340238  0.103830  0.199624  0.000000  0.000000  0.000000  0.136433   
38287  0.000000  0.000000  0.000000  0.000000  0.000000  0.341313  0.000000   
38288  0.120957  0.121906  0.000000  0.000000  0.215329  0.000000  0.000000   
38289  0.000000  0.243088  0.000000  0.000000  0.000000  0.000000  0.188654   

          конец   который   крупный  ..

In [90]:
res_df_v2 = result_df.drop(['ABRD', 'AFLT', 'ALRS', 'APTK', 'CHMF', 'GAZP',
        'GMKN', 'LKOH', 'LSRG', 'MGNT', 'MGTS', 'MOEX', 'MVID', 'NAUK', 'PIKK',
        'PLZL', 'RBCM', 'ROSN', 'RTKM', 'SBER', 'TATN', 'VTBR', 'finance',
        'gold', 'ferrous_metallurgy', 'oil_gas', 'non_ferrous_metallurgy',
        'electrical networks', 'telecom', 'it', 'real_estate', 'consume_retail',
        'tech_market', 'research'], axis=1)
print(res_df_v2)

           банк      весь     время  господин    данные  заявлять  компания  \
0      0.601004  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
1      0.000000  0.315523  0.000000  0.000000  0.000000  0.363113  0.244868   
2      0.000000  0.284154  0.000000  0.000000  0.000000  0.000000  0.373379   
3      0.000000  0.284154  0.000000  0.000000  0.000000  0.000000  0.373379   
4      0.000000  0.196424  0.000000  0.204815  0.204917  0.226051  0.258101   
...         ...       ...       ...       ...       ...       ...       ...   
38285  0.190141  0.000000  0.000000  0.000000  0.000000  0.000000  0.251806   
38286  0.340238  0.103830  0.199624  0.000000  0.000000  0.000000  0.136433   
38287  0.000000  0.000000  0.000000  0.000000  0.000000  0.341313  0.000000   
38288  0.120957  0.121906  0.000000  0.000000  0.215329  0.000000  0.000000   
38289  0.000000  0.243088  0.000000  0.000000  0.000000  0.000000  0.188654   

          конец   который   крупный  ...     слово 

In [91]:
train, test = result_df.iloc[:int(0.8*len(result_df)), :], result_df.iloc[int(0.8*len(result_df)):, :]
    
test_logg = []
train_logg = []


X_train = train.drop('class', axis=1)
X_test = test.drop('class', axis=1)

print(X_test)

y_train = train['class'].values
y_test = test['class'].values

print(y_test)
print(len(y_test))

           банк      весь     время  господин    данные  заявлять  компания  \
30632  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.393713   
30633  0.000000  0.000000  0.000000  0.000000  0.359445  0.000000  0.000000   
30634  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.393713   
30635  0.000000  0.000000  0.000000  0.000000  0.000000  0.306731  0.350221   
30636  0.000000  0.000000  0.000000  0.277887  0.278025  0.000000  0.000000   
...         ...       ...       ...       ...       ...       ...       ...   
38285  0.190141  0.000000  0.000000  0.000000  0.000000  0.000000  0.251806   
38286  0.340238  0.103830  0.199624  0.000000  0.000000  0.000000  0.136433   
38287  0.000000  0.000000  0.000000  0.000000  0.000000  0.341313  0.000000   
38288  0.120957  0.121906  0.000000  0.000000  0.215329  0.000000  0.000000   
38289  0.000000  0.243088  0.000000  0.000000  0.000000  0.000000  0.188654   

          конец   который   крупный  ...      свой 

In [92]:
# print(features_test[:10])
# print(features_test.shape)
# print(tfidf.get_feature_names_out())
# print(features_test.toarray())

# X_train = np.asarray(features_train.todense())
# X_test = np.asarray(features_test.todense())


# print(X_test[:10])
# print(len(X_test))
    

# print(y_test[:10])
# print(len(y_test))

param_grid = {'max_depth' : list(range(3,13))}

clf = RandomForestClassifier(random_state=123, criterion='entropy', n_jobs=-1)
gc = GridSearchCV(estimator=clf, param_grid=param_grid, cv=4, n_jobs=1, scoring='accuracy', verbose=2)
gc.fit(X_train, y_train)

preds_test = gc.predict(X_test)
# print(preds_test)
# print(len(preds_test))
preds_train = gc.predict(X_train)

preds_proba_test = gc.predict_proba(X_test)
print(preds_proba_test)
# print(len(preds_proba_test))
preds_proba_train = gc.predict_proba(X_train)
print(preds_proba_train)

#         test_logg.append(np.mean(y_test==preds_test))
#         train_logg.append(np.mean(y_train==preds_train))
print(np.mean(y_test==preds_test))
print(np.mean(y_train==preds_train))
print(gc.best_params_)


Fitting 4 folds for each of 10 candidates, totalling 40 fits
[CV] END ........................................max_depth=3; total time=   0.8s
[CV] END ........................................max_depth=3; total time=   0.7s
[CV] END ........................................max_depth=3; total time=   0.7s
[CV] END ........................................max_depth=3; total time=   0.8s
[CV] END ........................................max_depth=4; total time=   0.8s
[CV] END ........................................max_depth=4; total time=   0.8s
[CV] END ........................................max_depth=4; total time=   0.8s
[CV] END ........................................max_depth=4; total time=   0.8s
[CV] END ........................................max_depth=5; total time=   0.9s
[CV] END ........................................max_depth=5; total time=   0.9s
[CV] END ........................................max_depth=5; total time=   0.9s
[CV] END .......................................

# Попытка 3 (Используем взвешенный tf-idf + 1,2 n-граммы)

In [15]:
import ir
from collections import defaultdict

In [109]:
cur_df = index_df.loc[:5, ['message', 'class']]

count = defaultdict(lambda: {'pos': 0, 'neg': 0})

for index, row in cur_df.iterrows():
    
    words = ir.tokenize(row['message'])
    ngrams = set(words + ir.ngrams(words, 2))
#     print(ngrams)
#     break
    for ngram in ngrams:
        count[ngram]['pos'] += row['class'] == 1
        count[ngram]['neg'] += row['class'] == 0
    
print(count)

defaultdict(<function <lambda> at 0x000001E939989EE0>, {'35 чистый': {'pos': 0, 'neg': 2}, 'месторождение остаточный': {'pos': 0, 'neg': 2}, 'катарский': {'pos': 0, 'neg': 2}, 'сделка финансирование': {'pos': 0, 'neg': 1}, 'мсфо снижение': {'pos': 0, 'neg': 2}, '19 5': {'pos': 0, 'neg': 2}, 'ставка ндпи': {'pos': 0, 'neg': 2}, 'investment': {'pos': 0, 'neg': 2}, 'ндпи': {'pos': 0, 'neg': 2}, 'половина финансирование': {'pos': 0, 'neg': 1}, 'согласно заявление': {'pos': 0, 'neg': 1}, 'сделка покупка': {'pos': 0, 'neg': 1}, 'фонд катар': {'pos': 0, 'neg': 1}, 'катар пакет': {'pos': 0, 'neg': 1}, 'материал': {'pos': 3, 'neg': 6}, 'запас': {'pos': 0, 'neg': 2}, 'льгота': {'pos': 0, 'neg': 2}, '5 2': {'pos': 0, 'neg': 1}, 'также становиться': {'pos': 0, 'neg': 1}, 'соответствие': {'pos': 1, 'neg': 1}, '150 млн': {'pos': 0, 'neg': 2}, 'результат покупка': {'pos': 0, 'neg': 2}, 'банк': {'pos': 0, 'neg': 1}, 'заявление': {'pos': 0, 'neg': 1}, 'приобретение': {'pos': 0, 'neg': 2}, 'остаточный з

In [93]:
cur_df = index_df.loc[:, ['message', 'class']]
docs = []
for index, row in cur_df.iterrows():
    docs.append((row['class'], row['message']))

index = ir.SentimentIndex('delta', 'bogram')
index.get_class = lambda x: x[0]
index.get_text = lambda x: x[1]
index.build(docs)

x = []
y = []
for doc in docs:
    x.append(index.weight(index.features(doc)))
    y.append(doc[0])
    
# print(x)

In [94]:
print(len(x))
print(len(y))

38282
38282


In [95]:
count = []
for row in x:
    bal = 0
    for key, value in row.items():
        bal += value
    count.append(bal)
        
# print(count[:10])
# print(y[:10])
# print(count[990:])
# print(y[990:])

In [137]:
pip install liblinear-official

     -------------------------------------- 58.8/58.8 kB 516.5 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [140]:
from svm import SVM as Classifier
cl = Classifier()
cl.train(x, y)

In [148]:
docs_test = []
test_cur_df = index_df.loc[10000:12000, ['message']]

for ind, row in test_cur_df.iterrows():
    feats = index.weight(index.features(row['message']))
    docs_test.append(feats)

labels = cl.predict(docs_test)

for n in range(100):
    print('{0}.\t{1}'.format(n + 1, labels[n]))

Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
A

# Итерация 4 - продолжаем тактику (3)

In [96]:
df_iter_4 = cur_df.drop('message', axis=1)

In [100]:
ser = pd.Series(count)
df_iter_4_final = pd.concat([ser, df_iter_4], axis=1)
print(df_iter_4_final)

               0  class
0     -47.543185      0
1     -60.395658      0
2     -50.056234      0
3     -50.056234      0
4     -42.778148      0
...          ...    ...
38277 -83.855277      0
38278  -9.986679      1
38279 -24.818557      0
38280  -1.098612      0
38281 -43.566307      0

[38282 rows x 2 columns]


In [102]:
df_iter_4_final.rename(columns={0: 'score'}, inplace=True)
print(df_iter_4_final)

           score  class
0     -47.543185      0
1     -60.395658      0
2     -50.056234      0
3     -50.056234      0
4     -42.778148      0
...          ...    ...
38277 -83.855277      0
38278  -9.986679      1
38279 -24.818557      0
38280  -1.098612      0
38281 -43.566307      0

[38282 rows x 2 columns]


In [103]:
train, test = df_iter_4_final.iloc[:int(0.8*len(df_iter_4_final)), :], df_iter_4_final.iloc[int(0.8*len(df_iter_4_final)):, :]
    

X_train = train.drop('class', axis=1)
X_test = test.drop('class', axis=1)

print(X_test)

y_train = train['class'].values
y_test = test['class'].values

print(y_test)
print(len(y_test))

           score
30625 -48.515419
30626 -78.204298
30627 -33.159640
30628 -61.898453
30629 -42.633028
...          ...
38277 -83.855277
38278  -9.986679
38279 -24.818557
38280  -1.098612
38281 -43.566307

[7657 rows x 1 columns]
[1 1 1 ... 0 0 0]
7657


In [104]:
param_grid = {'max_depth' : list(range(1,13))}

clf = RandomForestClassifier(random_state=123, criterion='entropy', n_jobs=-1)
gc = GridSearchCV(estimator=clf, param_grid=param_grid, cv=4, n_jobs=1, scoring='accuracy', verbose=2)
gc.fit(X_train, y_train)

preds_test = gc.predict(X_test)
# print(preds_test)
# print(len(preds_test))
preds_train = gc.predict(X_train)

preds_proba_test = gc.predict_proba(X_test)
print(preds_proba_test)
# print(len(preds_proba_test))
preds_proba_train = gc.predict_proba(X_train)
print(preds_proba_train)

#         test_logg.append(np.mean(y_test==preds_test))
#         train_logg.append(np.mean(y_train==preds_train))
print(np.mean(y_test==preds_test))
print(np.mean(y_train==preds_train))
print(gc.best_params_)

Fitting 4 folds for each of 12 candidates, totalling 48 fits
[CV] END ........................................max_depth=1; total time=   0.2s
[CV] END ........................................max_depth=1; total time=   0.1s
[CV] END ........................................max_depth=1; total time=   0.1s
[CV] END ........................................max_depth=1; total time=   0.1s
[CV] END ........................................max_depth=2; total time=   0.2s
[CV] END ........................................max_depth=2; total time=   0.2s
[CV] END ........................................max_depth=2; total time=   0.2s
[CV] END ........................................max_depth=2; total time=   0.2s
[CV] END ........................................max_depth=3; total time=   0.3s
[CV] END ........................................max_depth=3; total time=   0.3s
[CV] END ........................................max_depth=3; total time=   0.3s
[CV] END .......................................

# Версия 5 - объединяем наработки

In [4]:
# import re
import os
# import warnings

# import numpy as np
import pandas as pd
from pymystem3 import Mystem
import nltk
from nltk.corpus import stopwords

In [5]:
mystem = Mystem()
nltk.download('stopwords')
swords = set(stopwords.words('russian'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zolgu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
from text_preprocess import lemm_file
roots = ['news_links/komersant_economics', 'news_links/komersant_finance', 'news_links/komersant_business']
# roots = ['news_links/komersant_finance']
for root in roots:
    files = sorted(os.listdir(f'{root}/raw_cites'))
    files = [f for f in files if f[0] != '.']
    c = 1

    for file in files:
        print(f'file {file} in work')
        df = lemm_file(root, f'/{file}', mystem, swords)
        if df is None:
            continue
        df.to_csv(f'{root}/prepared_cites_v5/{file[:-4]}.csv', index=False)
        print(f'{c} из {len(files)} готов')
        c += 1

file 1000.pkl in work
title ready
announce ready
text ready
1 из 126 готов
file 10000.pkl in work
title ready
announce ready
text ready
2 из 126 готов
file 10200.pkl in work
title ready
announce ready
text ready
3 из 126 готов
file 10400.pkl in work
title ready
announce ready
text ready
4 из 126 готов
file 10600.pkl in work
title ready
announce ready
text ready
5 из 126 готов
file 10800.pkl in work
title ready
announce ready
text ready
6 из 126 готов
file 11000.pkl in work
title ready
announce ready
text ready
7 из 126 готов
file 11200.pkl in work
title ready
announce ready
text ready
8 из 126 готов
file 11400.pkl in work
title ready
announce ready
text ready
9 из 126 готов
file 11600.pkl in work
title ready
announce ready
text ready
10 из 126 готов
file 11800.pkl in work
title ready
announce ready
text ready
11 из 126 готов
file 1200.pkl in work
title ready
announce ready
text ready
12 из 126 готов
file 12000.pkl in work
title ready
announce ready
text ready
13 из 126 готов
file 12200

title ready
announce ready
text ready
108 из 126 готов
file 6600.pkl in work
title ready
announce ready
text ready
109 из 126 готов
file 6800.pkl in work
title ready
announce ready
text ready
110 из 126 готов
file 7000.pkl in work
title ready
announce ready
text ready
111 из 126 готов
file 7200.pkl in work
title ready
announce ready
text ready
112 из 126 готов
file 7400.pkl in work
title ready
announce ready
text ready
113 из 126 готов
file 7600.pkl in work
title ready
announce ready
text ready
114 из 126 готов
file 7800.pkl in work
title ready
announce ready
text ready
115 из 126 готов
file 800.pkl in work
title ready
announce ready
text ready
116 из 126 готов
file 8000.pkl in work
title ready
announce ready
text ready
117 из 126 готов
file 8200.pkl in work
title ready
announce ready
text ready
118 из 126 готов
file 8400.pkl in work
title ready
announce ready
text ready
119 из 126 готов
file 8600.pkl in work
title ready
announce ready
text ready
120 из 126 готов
file 8800.pkl in work


title ready
announce ready
text ready
89 из 127 готов
file 3000.pkl in work
title ready
announce ready
text ready
90 из 127 готов
file 3200.pkl in work
title ready
announce ready
text ready
91 из 127 готов
file 3400.pkl in work
title ready
announce ready
text ready
92 из 127 готов
file 3600.pkl in work
title ready
announce ready
text ready
93 из 127 готов
file 3800.pkl in work
title ready
announce ready
text ready
94 из 127 готов
file 400.pkl in work
title ready
announce ready
text ready
95 из 127 готов
file 4000.pkl in work
title ready
announce ready
text ready
96 из 127 готов
file 4200.pkl in work
title ready
announce ready
text ready
97 из 127 готов
file 4400.pkl in work
title ready
announce ready
text ready
98 из 127 готов
file 4600.pkl in work
title ready
announce ready
text ready
99 из 127 готов
file 4800.pkl in work
title ready
announce ready
text ready
100 из 127 готов
file 5000.pkl in work
title ready
announce ready
text ready
101 из 127 готов
file 5200.pkl in work
title ready

title ready
announce ready
text ready
69 из 208 готов
file 22200.pkl in work
title ready
announce ready
text ready
70 из 208 готов
file 22400.pkl in work
title ready
announce ready
text ready
71 из 208 готов
file 22600.pkl in work
title ready
announce ready
text ready
72 из 208 готов
file 22800.pkl in work
title ready
announce ready
text ready
73 из 208 готов
file 23000.pkl in work
title ready
announce ready
text ready
74 из 208 готов
file 23200.pkl in work
title ready
announce ready
text ready
75 из 208 готов
file 23400.pkl in work
title ready
announce ready
text ready
76 из 208 готов
file 23600.pkl in work
title ready
announce ready
text ready
77 из 208 готов
file 23800.pkl in work
title ready
announce ready
text ready
78 из 208 готов
file 2400.pkl in work
title ready
announce ready
text ready
79 из 208 готов
file 24000.pkl in work
title ready
announce ready
text ready
80 из 208 готов
file 24200.pkl in work
title ready
announce ready
text ready
81 из 208 готов
file 24400.pkl in work


title ready
announce ready
text ready
175 из 208 готов
file 41400.pkl in work
title ready
announce ready
text ready
176 из 208 готов
file 41517.pkl in work
title ready
announce ready
text ready
177 из 208 готов
file 4200.pkl in work
title ready
announce ready
text ready
178 из 208 готов
file 4400.pkl in work
title ready
announce ready
text ready
179 из 208 готов
file 4600.pkl in work
title ready
announce ready
text ready
180 из 208 готов
file 4800.pkl in work
title ready
announce ready
text ready
181 из 208 готов
file 5000.pkl in work
title ready
announce ready
text ready
182 из 208 готов
file 5200.pkl in work
title ready
announce ready
text ready
183 из 208 готов
file 5400.pkl in work
title ready
announce ready
text ready
184 из 208 готов
file 5600.pkl in work
title ready
announce ready
text ready
185 из 208 готов
file 5800.pkl in work
title ready
announce ready
text ready
186 из 208 готов
file 600.pkl in work
title ready
announce ready
text ready
187 из 208 готов
file 6000.pkl in wor

In [7]:
from text_preprocess import komersant_to_standard_date
folders = ['news_links/komersant_' + t for t in ['economics', 'business', 'finance']]

komersant_to_standard_date(folders)

Обработано 100 из 126
Обработано 100 из 208
Обработано 200 из 208
Обработано 100 из 127


In [8]:
companies = {
    'ABRD': 'Абрау-Дюрсо ПАО ао', 
    'AFLT': 'Аэрофлот-росс.авиалин(ПАО)ао', 
    'ALRS': 'АЛРОСА ПАО ао',
    'APTK': 'ПАО "Аптечная сеть 36,6" ао',
    'CHMF': 'Северсталь (ПАО)ао',
    'GAZP': '"Газпром" (ПАО) ао', 
    'GMKN': 'ГМК "Нор.Никель" ПАО ао', 
    'LKOH': 'НК ЛУКОЙЛ (ПАО) - ао',
    'LSRG': 'Группа ЛСР ПАО ао',
    'MGNT': '"Магнит" ПАО ао',
    'MGTS': '"ПАО "МГТС" ао',
    'MOEX': 'ПАО Московская Биржа', 
    'MVID': '"М.видео" ПАО ао',
    'NAUK': 'НПО Наука ао',
    'PIKK': 'ПИК СЗ (ПАО) ао', 
    'PLZL': 'Полюс ПАО ао',
    'RBCM': 'ГК РБК ПАО ао',
    'ROSN': 'ПАО НК Роснефть',
    'RTKM': 'Ростелеком (ПАО) ао.',  
    'SBER': 'Сбербанк России ПАО ао',
    'TATN': 'ПАО "Татнефть" ао',
    'VTBR': 'ао ПАО Банк ВТБ'
    }

In [9]:
for_regex = {
    'ABRD': ['абрау-дюрсо', 'абрау', 'abrd'],
    'ALRS': ['алроса', 'alrs'], 
    'AFLT': ['аэрофлот', 'aflt'],
    'APTK': ['36.6', '36,6', 'aptk'],
    'CHMF': ['cеверсталь', 'chmf'],
    'GAZP': ['газпром', 'gazp'], 
    'GMKN': ['норникель', 'нор никель', 'норильский никель', 'gmkn'],
    'LKOH': ['лукойл', 'lkoh'],
    'LSRG': ['лср', 'lsr', 'lsrg'], 
    'MGNT': ['магнит', 'magnit', 'mgnt'],
    'MGTS': ['мгтс', 'московская городская телефонная сеть', 'mgts'],
    'MOEX': ['московский биржа', 'мосбиржа', 'мос биржа', 'мос. биржа', 'moex'],
    'MVID': ['мвидео', 'м. видео', 'м.видео', 'mvideo', 'm. video', 'm.video', 'mvid'],
    'NAUK': ['нпо наука', 'nauk'],
    'PIKK': ['pik', 'пик', 'первая ипотечная компания', 'pikk'], 
    'PLZL': ['plzl', 'полюс золото'],
    'RBCM': ['rbc', 'рбк', 'росбизнесконсалтинг', 'ragr'],
    'ROSN': ['роснефть', 'рос нефть', 'rosneft', 'rosn'],
    'RTKM': ['ростелеком', 'рос телеком', 'rtkm'],  
    'SBER': ['/^сбер$/', 'сбербанк', 'sber'],   
    'TATN': ['tatn', 'tatneft', 'татнефть'], 
    'VTBR': ['втб', 'vtb', 'vtbr']
}

In [10]:
from text_preprocess import add_target_v5
folders = ['news_links/komersant_' + t for t in ['economics', 'business', 'finance']]
# folders = ['news_links/komersant_' + t for t in ['economics']]

for folder in folders:
    add_target_v5(folder, companies)
    print(f'{folder} готов')

file gathered
ABRD готов
AFLT готов
ALRS готов
APTK готов
CHMF готов
GAZP готов
GMKN готов
LKOH готов
LSRG готов
MGNT готов
MGTS готов
MOEX готов
MVID готов
NAUK готов
PIKK готов
PLZL готов
RBCM готов
ROSN готов
RTKM готов
SBER готов
TATN готов
VTBR готов
news_links/komersant_economics готов
file gathered
ABRD готов
AFLT готов
ALRS готов
APTK готов
CHMF готов
GAZP готов
GMKN готов
LKOH готов
LSRG готов
MGNT готов
MGTS готов
MOEX готов
MVID готов
NAUK готов
PIKK готов
PLZL готов
RBCM готов
ROSN готов
RTKM готов
SBER готов
TATN готов
VTBR готов
news_links/komersant_business готов
file gathered
ABRD готов
AFLT готов
ALRS готов
APTK готов
CHMF готов
GAZP готов
GMKN готов
LKOH готов
LSRG готов
MGNT готов
MGTS готов
MOEX готов
MVID готов
NAUK готов
PIKK готов
PLZL готов
RBCM готов
ROSN готов
RTKM готов
SBER готов
TATN готов
VTBR готов
news_links/komersant_finance готов


In [11]:
folders = ['news_links/komersant_' + t for t in ['economics','business', 'finance']]
end_df = pd.DataFrame()
for folder in folders:
    print(folder)
    df = pd.read_csv(f'{folder}/targeted_v5.csv')
    end_df = pd.concat([end_df, df])
 
end_df.sort_values('date', ignore_index=True, inplace=True)
print(len(end_df))
end_df.to_csv(f'news_links/final_v5.csv', index=False)

news_links/komersant_economics
news_links/komersant_business
news_links/komersant_finance
91948


In [12]:
import pandas as pd
tmp = pd.read_csv(f'news_links/final_v5.csv', sep=',')
print(tmp.isna().sum())

link                 0
date                 0
title                9
announce             0
text               320
title origin         0
announce origin      0
text origin        320
ABRD close prev      8
ABRD close           0
AFLT close prev      8
AFLT close           0
ALRS close prev      8
ALRS close           0
APTK close prev      8
APTK close           0
CHMF close prev      8
CHMF close           0
GAZP close prev      8
GAZP close           0
GMKN close prev      8
GMKN close           0
LKOH close prev      8
LKOH close           0
LSRG close prev      8
LSRG close           0
MGNT close prev      8
MGNT close           0
MGTS close prev      8
MGTS close           0
MOEX close prev      8
MOEX close           0
MVID close prev      8
MVID close           0
NAUK close prev      8
NAUK close           0
PIKK close prev      8
PIKK close           0
PLZL close prev      8
PLZL close           0
RBCM close prev      8
RBCM close           0
ROSN close prev      8
ROSN close 

In [13]:
tmp = tmp.dropna()
tmp.drop_duplicates(inplace=True)
tmp.sort_values('date', ignore_index=True, inplace=True)
print(len(tmp))
print(tmp.isna().sum())
tmp.to_csv(f'news_links/final_v5.csv', index=False)

90214
link               0
date               0
title              0
announce           0
text               0
title origin       0
announce origin    0
text origin        0
ABRD close prev    0
ABRD close         0
AFLT close prev    0
AFLT close         0
ALRS close prev    0
ALRS close         0
APTK close prev    0
APTK close         0
CHMF close prev    0
CHMF close         0
GAZP close prev    0
GAZP close         0
GMKN close prev    0
GMKN close         0
LKOH close prev    0
LKOH close         0
LSRG close prev    0
LSRG close         0
MGNT close prev    0
MGNT close         0
MGTS close prev    0
MGTS close         0
MOEX close prev    0
MOEX close         0
MVID close prev    0
MVID close         0
NAUK close prev    0
NAUK close         0
PIKK close prev    0
PIKK close         0
PLZL close prev    0
PLZL close         0
RBCM close prev    0
RBCM close         0
ROSN close prev    0
ROSN close         0
RTKM close prev    0
RTKM close         0
SBER close prev    0
SBER cl

In [14]:
from text_preprocess import add_company_v5
add_company_v5(for_regex, companies)

done


In [15]:
for_regex_industry = {
    'finance': ['ипотека', 'sber', '/^сбер$/', 'сбербанк', 'банк', 'кредит', 'депозит', 'мкб', 'московский кредитный банк', 'cbom', 'vtbr', 'втб', 'финансовый', 'финансы', 'биржа', 'moex', 'tcsg', 'тинькофф', 'ценные бумаги', 'ценная бумага', 'облигация', 'облигации', 'паевый', 'фонд', 'котировка'],
    'gold': ['poly', 'plzl', 'полиметалл', 'полюс золото', 'золото', 'pogr', 'petropavlovsk plc', 'petropavlovsk'],
    'ferrous_metallurgy': ['железо', 'сталь', 'северсталь', 'чугун', 'ммк', 'магнитогорский металлургический комбинат', 'magn', 'северсталь', 'chmf', 'nlmk', 'новолипецкий металлургический комбинат', 'металл'],
    'oil_gas':['gazp', 'газпром', 'rosn', 'роснефть', 'сургутнефтегаз', 'sngs', 'газ', 'нефть', 'нефтепровод', 'газопровод', 'татнефть', 'tatn', 'лукойл', 'lkoh', 'транснефть', 'trnfp', 'новатэк', 'nvtk'],
    'non_ferrous_metallurgy': ['rual', 'rusal', 'uc rusal','русал', 'gmkn', 'норникель', 'норильский никель', 'ниель', 'алюминий', 'палладий', 'олово', 'медь', 'глинозем'],
    'electrical networks': ['интер рао', 'irao', 'русгидро', 'hydr', 'элетричество', 'гэс', 'тэц', 'энергетика', 'фск еэс', 'fees'],
    'telecom': ['мтс', 'mtss', 'ростелеком', 'rtkm', 'сеть', 'связь', 'интернет', 'провайдер'],
    'it': ['машинный обучение', 'цифровой технология', 'облако', 'вк', 'мэйл', 'вконтакте', 'mail', 'vk', 'vkco', 'яндекс', 'yandex', 'yndx'],
    'real_estate': ['недвижимость', 'новостройка', 'ипотека', 'лср', 'lsrg', 'пик', 'pikk'],
    'consume_retail': ['напиток', 'вода', 'абрау', 'дюрсо', 'abrd', 'алкоголь', 'вино', 'еда', 'пищевой', 'пища', 'магнит', 'перекресток', 'пятерочка', 'продукт', 'магазин'],
    'tech_market': ['м.видео', 'mvideo', 'm. video', 'dns', 'citilink', 'смартфон', 'ноутбук', 'телевизор', 'бытовой', 'техника'],
    'research': ['исследование', 'наука', 'научный', 'nauk', 'нпо'],
}

industries = for_regex_industry.keys()

In [16]:
from text_preprocess import add_industry_v5
add_industry_v5(industries, for_regex_industry)

done


In [17]:
import pandas as pd
df = pd.read_csv(f'news_links/final_v5.csv')
print(len(df))
print(df.value_counts('finance'))

90214
finance
True     49319
False    40895
dtype: int64


In [2]:
import pandas as pd


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "D:\PROgrams_for_teach\anaconda\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "D:\PROgrams_for_teach\anaconda\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "D:\PROgrams_for_teach\anaconda\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "D:\PROgrams_for_teach\anaconda\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
    app.start()
  File "D:\PROgrams_fo

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "D:\PROgrams_for_teach\anaconda\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "D:\PROgrams_for_teach\anaconda\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "D:\PROgrams_for_teach\anaconda\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "D:\PROgrams_for_teach\anaconda\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
    app.start()
  File "D:\PROgrams_fo

AttributeError: _ARRAY_API not found

In [3]:
news = pd.read_csv(f'news_links/final_v5.csv')

# news['message'] = news['title'] + ' ' + news['announce'] + ' ' + news['text']
    
news.drop(columns=['link', 'title', 'announce', 'text'], inplace=True)
print(news.isna().sum())

date               0
title origin       0
announce origin    0
text origin        0
ABRD close prev    0
                  ..
it                 0
real_estate        0
consume_retail     0
tech_market        0
research           0
Length: 82, dtype: int64


In [5]:
print(news[:2])

                  date                                       title origin  \
0  2017-01-03 22:22:00  glencore и катарский суверенный фонд закрыли с...   
1  2017-01-03 22:47:02  дональд трамп раскритиковал general motors за ...   

  announce origin                                        text origin  \
0     no announce  швейцарский нефтетрейдер glencore и катарский ...   
1     no announce  избранный президент сша дональд трамп раскрити...   

   ABRD close prev  ABRD close  AFLT close prev  AFLT close  ALRS close prev  \
0             99.5        99.5           151.68       148.4           100.54   
1             99.5        99.5           151.68       148.4           100.54   

   ALRS close  ...  ferrous_metallurgy  oil_gas  non_ferrous_metallurgy  \
0       101.1  ...               False     True                   False   
1       101.1  ...               False    False                   False   

   electrical networks  telecom     it  real_estate  consume_retail  \
0            

In [6]:
print(news.columns)

Index(['date', 'title origin', 'announce origin', 'text origin',
       'ABRD close prev', 'ABRD close', 'AFLT close prev', 'AFLT close',
       'ALRS close prev', 'ALRS close', 'APTK close prev', 'APTK close',
       'CHMF close prev', 'CHMF close', 'GAZP close prev', 'GAZP close',
       'GMKN close prev', 'GMKN close', 'LKOH close prev', 'LKOH close',
       'LSRG close prev', 'LSRG close', 'MGNT close prev', 'MGNT close',
       'MGTS close prev', 'MGTS close', 'MOEX close prev', 'MOEX close',
       'MVID close prev', 'MVID close', 'NAUK close prev', 'NAUK close',
       'PIKK close prev', 'PIKK close', 'PLZL close prev', 'PLZL close',
       'RBCM close prev', 'RBCM close', 'ROSN close prev', 'ROSN close',
       'RTKM close prev', 'RTKM close', 'SBER close prev', 'SBER close',
       'TATN close prev', 'TATN close', 'VTBR close prev', 'VTBR close',
       'ABRD', 'AFLT', 'ALRS', 'APTK', 'CHMF', 'GAZP', 'GMKN', 'LKOH', 'LSRG',
       'MGNT', 'MGTS', 'MOEX', 'MVID', 'NAUK', 'PIKK'

## Тест отдельных компаний

In [5]:
# companies = {
#     'GAZP': '"Газпром" (ПАО) ао', 
#     }

In [9]:
df = pd.DataFrame()
for company in companies:
    comp = news[news[company] == True].copy()
    comp['close'] = comp[f'{company} close'] - comp[f'{company} close prev']
    comp = comp.loc[:, ['title origin', 'announce origin', 'text origin', 'close', 'date',
        'ABRD', 'AFLT', 'ALRS', 'APTK', 'CHMF', 'GAZP',
        'GMKN', 'LKOH', 'LSRG', 'MGNT', 'MGTS', 'MOEX', 'MVID', 'NAUK', 'PIKK',
        'PLZL', 'RBCM', 'ROSN', 'RTKM', 'SBER', 'TATN', 'VTBR', 'finance',
        'gold', 'ferrous_metallurgy', 'oil_gas', 'non_ferrous_metallurgy',
        'electrical networks', 'telecom', 'it', 'real_estate', 'consume_retail',
        'tech_market', 'research']]
    df = pd.concat([df, comp], axis=0)
df.drop_duplicates(inplace=True)
df.sort_values('date', ignore_index=True, inplace=True)


# df = df.loc[(df['date'] <= '2020-12-31')]
            
            
print(df)

                                            title origin  \
0      glencore и катарский суверенный фонд закрыли с...   
1      банк intesa обеспечил половину сделки по прива...   
2      иран допустил «газпром» и лукойл к участию в т...   
3      иран допустил «газпром» и лукойл к участию в т...   
4      «роснефтегаз» подтвердил закрытие сделки по пр...   
...                                                  ...   
38257  убыток спб биржи по рсбу снизился в 2024 году ...   
38258  совет директоров «эл5-энерго» рекомендовал не ...   
38259  ктк приостановил работу двух причальных устрой...   
38260  ктк приостановил работу двух причальных устрой...   
38261                                трамплин для золота   

                                         announce origin  \
0                                            no announce   
1                                            no announce   
2                                            no announce   
3                                      

In [12]:
print(len(df))

38262


In [14]:
df = df[df['close'] != 0]

In [15]:
print(len(df))

37595


In [10]:
print(df.value_counts('announce origin'))

announce origin
no announce                                                                         20429
мнения и прогнозы аналитиков о том, как будет вести себя рубль                        280
мониторинг государственных финансов                                                    26
инфографика                                                                            26
что будет с рублем, долларом и нефтью                                                  17
                                                                                    ...  
14 тысяч цистерн уедут к «нефтетранссервису»                                            1
100% нпф «телеком-союз» могут консолидировать компании сергея сударикова                1
10 крупнейших госконтрактов и заказчиков 2019 года                                      1
$325 млн пойдут в благотворительный фонд и на урегулирование юридических проблем        1
"роснефть" увеличит поставки в китай                                                

In [46]:
# df['class'] = df['close'] > 0
# df = df.replace({True: 1, False: 0})
# df.drop(columns=['close', 'date'], inplace=True)
# print(df)

                                           title origin  \
0     \r\n        Иран допустил «Газпром» и ЛУКОЙЛ к...   
1     \r\n        «Нафтогаз» назвал способ сохранени...   
2     \r\n                «Газпром» увеличил заявку ...   
3     \r\n        «Газпром» установил новый рекорд п...   
4     \r\n        «Газпром» трижды с начала года заф...   
...                                                 ...   
5642  \r\n                Фондовому рынку арестовали...   
5643  \r\n                Ariston задала программу в...   
5644  \r\n                Путин разрешил иностранцам...   
5645  \r\n                «Газпром» существенно увел...   
5646  \r\n                Газ для потребителей в Гер...   

                                                message  ABRD  AFLT  ALRS  \
0     иран допускать газпром лукойл участие тендер н...     0     0     0   
1     нафтогаз называть способ сохранение украина тр...     0     0     0   
2     газпром увеличивать заявка транзит газ украина...     

In [11]:
import ir
from collections import defaultdict

In [30]:
import ir
title_origin_df = df.loc[:37095, ['title origin', 'close']]
# title_origin_df = df.loc[:, ['title origin', 'close']]
# print(len(title_origin_df))
docs = []
for ind, row in title_origin_df.iterrows():
    docs.append((row['close'], row['title origin']))

# print(docs[:5])
# print(type(docs[0][0]))

index = ir.SentimentIndex('delta', 'bogram')
index.get_class = lambda x: x[0]
index.get_text = lambda x: x[1]
index.build(docs)

x = []
y = []
for doc in docs:
    x.append(index.weight(index.features(doc)))
    y.append(doc[0])
   
# Тест на влияние тестовых данных на алгоритм тональности
title_origin_end = df.loc[len(title_origin_df):, ['title origin', 'close']]
# print(len(title_origin_end))

docs2 = []
vals = []
for ind, row in title_origin_end.iterrows():
    docs2.append((0, row['title origin']))
    vals.append(row['close'])
    
# print(docs2[:5])

i = 0
for doc in docs2:
    x.append(index.weight(index.features(doc)))
    y.append(vals[i])
    i += 1


In [12]:
index.save('test.index')
print('sentiment analyzer saved')
# index = ir.SentimentIndex.load('test.index', 'delta', 'bogram')

sentiment analyzer saved


In [15]:
index = ir.SentimentIndex.load('test.index', 'delta', 'bogram')
index.get_text = lambda x: x[1]

title_origin_df = df.loc[38000:, ['title origin', 'close']]
# print(len(title_origin_df))
docs = []
for ind, row in title_origin_df.iterrows():
    docs.append((row['close'], row['title origin']))
    

x = []
y = []
for doc in docs:
    x.append(index.weight(index.features(doc)))
    y.append(doc[0])


In [20]:
print("Размер датасета: ", len(x))
# print(len(y))
print("Значения взвешенного TF-IDF: ", x[-3])
print("Изменение цены: ", round(y[-3], 2))

Размер датасета:  38258
Значения взвешенного TF-IDF:  {'ктк': -3.6109179126442243, 'приостановил': -4.418840607796598, 'работу': -5.043425116919247, 'двух': -4.77912349311153, 'причальных': 0, 'устройств': -1.0986122886681098, 'на': -9.66896702819858, 'ктк приостановил': 0, 'приостановил работу': -2.3978952727983707, 'работу двух': 0, 'двух причальных': 0, 'причальных устройств': 0, 'устройств на': 0}
Изменение цены:  -11.6


In [31]:
ml_ready_df = pd.DataFrame()

for row in x[36596:37095]:
    ml_ready_df = pd.concat([ml_ready_df, pd.DataFrame([row])], ignore_index=True)
    
cols = ml_ready_df.columns.tolist()
print(type(cols))

for row in x[37095:]:
    ml_ready_df = pd.concat([ml_ready_df, pd.DataFrame([row])], ignore_index=True).loc[:, cols]


print(ml_ready_df)

<class 'list'>
           цб  установил      курс   доллара      выше       100  \
0    -7.41397  -4.418841 -7.186144 -6.918695 -5.720312 -5.849325   
1         NaN        NaN       NaN       NaN       NaN       NaN   
2         NaN        NaN       NaN       NaN       NaN       NaN   
3         NaN        NaN       NaN       NaN       NaN       NaN   
4         NaN        NaN       NaN       NaN       NaN       NaN   
...       ...        ...       ...       ...       ...       ...   
1657      NaN        NaN       NaN       NaN       NaN       NaN   
1658      NaN        NaN       NaN       NaN       NaN       NaN   
1659      NaN        NaN       NaN       NaN       NaN       NaN   
1660      NaN        NaN       NaN       NaN       NaN       NaN   
1661      NaN        NaN       NaN       NaN       NaN       NaN   

      цб установил  установил курс  курс доллара  доллара выше  ...  \
0        -2.833213        -2.70805     -6.792344      -2.70805  ...   
1              NaN        

In [11]:
cols = ml_ready_df.columns.tolist()
print(cols)
print(len(cols))
with open("columns.txt", "w", encoding='UTF-8') as file:
    file.write("\n".join(cols))
print('done')

['сбербанк', 'повысил', 'ставки', 'по', 'флагманской', 'линейке', 'сбербанк повысил', 'повысил ставки', 'ставки по', 'по флагманской', 'флагманской линейке', 'россияне', 'избегают', 'россияне избегают', 'утро', 'вечера', 'утро вечера', 'мвд', 'предупредило', 'об', 'атаках', 'мошенников', 'от', 'лица', 'мвд предупредило', 'предупредило об', 'об атаках', 'атаках мошенников', 'мошенников от', 'от лица', 'индекс', 'этажом', 'индекс этажом', 'энергия', 'растрачивается', 'энергия растрачивается', 'растрачивается по', 'в', 'бондах', 'как', 'в бондах', 'бондах как', 'как в', 'нефть', 'не', 'нефть не', 'да', 'тут-то', 'да не', 'не тут-то', 'науку', 'спросят', 'о', 'науку спросят', 'спросят о', 'аэрофлот', 'за', '2024', 'год', 'получил', '22', 'млрд', 'руб', 'прибыли', 'против', 'убытка', 'годом', 'аэрофлот за', 'за 2024', '2024 год', 'год получил', 'получил 22', '22 млрд', 'млрд руб', 'руб прибыли', 'прибыли против', 'против убытка', 'убытка годом', 'северсталь', 'решила', 'выплачивать', 'дивид

In [13]:
print(len(cols))

2246


In [32]:
sc = pd.Series(y[36596:])

ml_ready_df = pd.concat([ml_ready_df, sc], axis=1)
ml_ready_df.rename(columns={0: 'close'}, inplace=True)
print(ml_ready_df)

           цб  установил      курс   доллара      выше       100  \
0    -7.41397  -4.418841 -7.186144 -6.918695 -5.720312 -5.849325   
1         NaN        NaN       NaN       NaN       NaN       NaN   
2         NaN        NaN       NaN       NaN       NaN       NaN   
3         NaN        NaN       NaN       NaN       NaN       NaN   
4         NaN        NaN       NaN       NaN       NaN       NaN   
...       ...        ...       ...       ...       ...       ...   
1657      NaN        NaN       NaN       NaN       NaN       NaN   
1658      NaN        NaN       NaN       NaN       NaN       NaN   
1659      NaN        NaN       NaN       NaN       NaN       NaN   
1660      NaN        NaN       NaN       NaN       NaN       NaN   
1661      NaN        NaN       NaN       NaN       NaN       NaN   

      цб установил  установил курс  курс доллара  доллара выше  ...  \
0        -2.833213        -2.70805     -6.792344      -2.70805  ...   
1              NaN             NaN       

In [43]:
# ml_ready_df.drop('title score', axis=1)

,glencore,и,катарский,суверенный,фонд,закрыли,сделку,по,покупке,доли,...,взносов,повышение,сми в,в правительстве,правительстве согласовали,согласовали снижение,снижение страховых,страховых взносов,взносов и,и повышение
0,-3.970292,-8.604288,-2.564949,-2.197225,-4.574711,-3.713572,-5.187386,-8.669571,-5.332719,-5.433722,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-8.669571,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,-8.604288,NaN,NaN,NaN,NaN,NaN,-8.669571,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,-8.604288,NaN,NaN,NaN,NaN,NaN,-8.669571,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-8.669571,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
496,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
497,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
498,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [99]:
# count = []
# for row in x:
#     bal = 0
#     for key, value in row.items():
#         bal += value
#     count.append(bal)

# ser = pd.Series(count)
# df = pd.concat([ser, df], axis=1)

# df.drop(columns=['title origin'], inplace=True)
# print(df)

                0                                            message   close  \
0      -87.154834  glencore катарский суверенный фонд закрывать с...  -13.10   
1      -50.656097  банк intesa обеспечивать половина сделка прива...  -13.10   
2      -87.224337  иран допускать газпром лукойл участие тендер н...  -95.50   
3      -87.224337  иран допускать газпром лукойл участие тендер н...   -0.26   
4      -66.393411  роснефтегаз подтверждать закрытие сделка прода...   -7.40   
...           ...                                                ...     ...   
38263 -117.400995  убыток спб биржа рсба снижаться 2024 год 10 6 ...   -2.21   
38264  -85.140840  совет директор эл5 энерго рекомендовать выплач...  -32.00   
38265  -42.233408  ктк приостанавливать работа причальный устройс...  -11.60   
38266  -42.233408  ктк приостанавливать работа причальный устройс... -110.50   
38267  -10.014447  трамплин золото котировка золото мировой спот ...   -3.74   

                      date   ABRD   AFL

In [100]:
df.rename(columns={0: 'title score'}, inplace=True)
print(df)

       title score                                            message   close  \
0       -87.154834  glencore катарский суверенный фонд закрывать с...  -13.10   
1       -50.656097  банк intesa обеспечивать половина сделка прива...  -13.10   
2       -87.224337  иран допускать газпром лукойл участие тендер н...  -95.50   
3       -87.224337  иран допускать газпром лукойл участие тендер н...   -0.26   
4       -66.393411  роснефтегаз подтверждать закрытие сделка прода...   -7.40   
...            ...                                                ...     ...   
38263  -117.400995  убыток спб биржа рсба снижаться 2024 год 10 6 ...   -2.21   
38264   -85.140840  совет директор эл5 энерго рекомендовать выплач...  -32.00   
38265   -42.233408  ктк приостанавливать работа причальный устройс...  -11.60   
38266   -42.233408  ктк приостанавливать работа причальный устройс... -110.50   
38267   -10.014447  трамплин золото котировка золото мировой спот ...   -3.74   

                      date 

In [101]:
message_df = df.loc[:, ['message', 'close']]
docs = []
for index, row in message_df.iterrows():
    docs.append((row['close'], row['message']))

index = ir.SentimentIndex('delta', 'bogram')
index.get_class = lambda x: x[0]
index.get_text = lambda x: x[1]
index.build(docs)

x = []
y = []
for doc in docs:
    x.append(index.weight(index.features(doc)))
    y.append(doc[0])

In [27]:
print(len(x))
print(len(y))
print(x[:2])
print(y[:2])

38262
38262
[{'glencore': -3.970291913552122, 'и': -8.6020856584342, 'катарский': -2.5649493574615367, 'суверенный': -2.1972245773362196, 'фонд': -4.574710978503383, 'закрыли': -3.713572066704308, 'сделку': -5.187385805840755, 'по': -8.667163717992533, 'покупке': -5.332718793265369, 'доли': -5.43372200355424, 'в': -9.78318258949248, 'glencore и': -2.3978952727983707, 'и катарский': -1.0986122886681098, 'катарский суверенный': -1.9459101490553135, 'суверенный фонд': -1.9459101490553135, 'фонд закрыли': -1.0986122886681098, 'закрыли сделку': -2.1972245773362196, 'сделку по': -4.672828834461907, 'по покупке': -4.343805421853684, 'покупке доли': -2.70805020110221, 'доли в': -4.709530201312334}, {'банк': -5.926926025970411, 'intesa': -3.044522437723423, 'обеспечил': -1.0986122886681098, 'половину': -3.7612001156935624, 'сделки': -5.493061443340548, 'по': -8.667163717992533, 'приватизации': -4.859812404361672, 'пакета': -3.7612001156935624, 'банк intesa': -2.1972245773362196, 'intesa обеспеч

In [103]:
count = []
for row in x:
    bal = 0
    for key, value in row.items():
        bal += value
    count.append(bal)

ser = pd.Series(count)
df = pd.concat([ser, df], axis=1)

df.drop(columns=['message'], inplace=True)
print(df)

                 0  title score   close                 date   ABRD   AFLT  \
0     -1334.124617   -87.154834  -13.10  2017-01-03 22:22:00  False  False   
1     -1462.319898   -50.656097  -13.10  2017-01-04 11:55:54  False  False   
2     -1050.712625   -87.224337  -95.50  2017-01-04 14:05:44  False  False   
3     -1050.712625   -87.224337   -0.26  2017-01-04 14:05:44  False  False   
4     -1642.588585   -66.393411   -7.40  2017-01-04 19:14:54  False  False   
...            ...          ...     ...                  ...    ...    ...   
38263 -2059.468687  -117.400995   -2.21  2025-03-31 16:43:44  False  False   
38264 -1811.259448   -85.140840  -32.00  2025-03-31 18:26:23  False  False   
38265 -1821.884833   -42.233408  -11.60  2025-03-31 20:36:19  False  False   
38266 -1821.884833   -42.233408 -110.50  2025-03-31 20:36:19  False  False   
38267 -6339.260150   -10.014447   -3.74  2025-03-31 20:58:57  False  False   

        ALRS   APTK   CHMF   GAZP  ...  ferrous_metallurgy  oil

In [104]:
df.rename(columns={0: 'message score'}, inplace=True)
print(df)

       message score  title score   close                 date   ABRD   AFLT  \
0       -1334.124617   -87.154834  -13.10  2017-01-03 22:22:00  False  False   
1       -1462.319898   -50.656097  -13.10  2017-01-04 11:55:54  False  False   
2       -1050.712625   -87.224337  -95.50  2017-01-04 14:05:44  False  False   
3       -1050.712625   -87.224337   -0.26  2017-01-04 14:05:44  False  False   
4       -1642.588585   -66.393411   -7.40  2017-01-04 19:14:54  False  False   
...              ...          ...     ...                  ...    ...    ...   
38263   -2059.468687  -117.400995   -2.21  2025-03-31 16:43:44  False  False   
38264   -1811.259448   -85.140840  -32.00  2025-03-31 18:26:23  False  False   
38265   -1821.884833   -42.233408  -11.60  2025-03-31 20:36:19  False  False   
38266   -1821.884833   -42.233408 -110.50  2025-03-31 20:36:19  False  False   
38267   -6339.260150   -10.014447   -3.74  2025-03-31 20:58:57  False  False   

        ALRS   APTK   CHMF   GAZP  ... 

In [59]:
test_df = df.loc[:, ['message score', 'class']]

In [105]:
test_df = df
test_df['class'] = test_df['close'] > 0
test_df.drop(columns=['close', 'date'], inplace=True)
test_df = test_df.replace({True: 1, False: 0})

# print(test_df)

## 6 итерация - каждый признак - столбец

In [33]:
test_df = ml_ready_df.copy()
test_df['class'] = test_df['close'] > 0
test_df.drop(columns=['close'], inplace=True)

In [34]:
test_df = test_df.replace({True: 1, False: 0})
test_df = test_df.fillna(0)
print(test_df)

           цб  установил      курс   доллара      выше       100  \
0    -7.41397  -4.418841 -7.186144 -6.918695 -5.720312 -5.849325   
1     0.00000   0.000000  0.000000  0.000000  0.000000  0.000000   
2     0.00000   0.000000  0.000000  0.000000  0.000000  0.000000   
3     0.00000   0.000000  0.000000  0.000000  0.000000  0.000000   
4     0.00000   0.000000  0.000000  0.000000  0.000000  0.000000   
...       ...        ...       ...       ...       ...       ...   
1657  0.00000   0.000000  0.000000  0.000000  0.000000  0.000000   
1658  0.00000   0.000000  0.000000  0.000000  0.000000  0.000000   
1659  0.00000   0.000000  0.000000  0.000000  0.000000  0.000000   
1660  0.00000   0.000000  0.000000  0.000000  0.000000  0.000000   
1661  0.00000   0.000000  0.000000  0.000000  0.000000  0.000000   

      цб установил  установил курс  курс доллара  доллара выше  ...  \
0        -2.833213        -2.70805     -6.792344      -2.70805  ...   
1         0.000000         0.00000      0

C:\Users\zolgu\AppData\Local\Temp\ipykernel_4788\3809364151.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_df = test_df.replace({True: 1, False: 0})


In [35]:
train, test = test_df.iloc[:int(0.5*len(test_df)), :], test_df.iloc[int(0.5*len(test_df)):, :]
    

X_train = train.drop('class', axis=1)
X_test = test.drop('class', axis=1)

print(X_test)

y_train = train['class'].values
y_test = test['class'].values

print(y_test)
print(len(y_test))

       цб  установил      курс   доллара  выше       100  цб установил  \
831   0.0        0.0  0.000000  0.000000   0.0  0.000000           0.0   
832   0.0        0.0  0.000000  0.000000   0.0  0.000000           0.0   
833   0.0        0.0  0.000000  0.000000   0.0  0.000000           0.0   
834   0.0        0.0  0.000000  0.000000   0.0  0.000000           0.0   
835   0.0        0.0 -7.186144 -6.918695   0.0 -5.849325           0.0   
...   ...        ...       ...       ...   ...       ...           ...   
1657  0.0        0.0  0.000000  0.000000   0.0  0.000000           0.0   
1658  0.0        0.0  0.000000  0.000000   0.0  0.000000           0.0   
1659  0.0        0.0  0.000000  0.000000   0.0  0.000000           0.0   
1660  0.0        0.0  0.000000  0.000000   0.0  0.000000           0.0   
1661  0.0        0.0  0.000000  0.000000   0.0  0.000000           0.0   

      установил курс  курс доллара  доллара выше  ...  по выкупу  \
831              0.0      0.000000         

In [26]:
from sklearn.ensemble import RandomForestClassifier
# from lightgbm import LGBMClassifier
# from sklearn import svm
from sklearn.model_selection import GridSearchCV
import os
import pandas as pd
import numpy as np
import joblib

In [40]:
param_grid = {'max_depth' : list(range(4,13))}


clf_forest = RandomForestClassifier(criterion='entropy', n_jobs=-1)
# clf2_boost = LGBMClassifier(random_state=123, n_jobs=-1)
# clf3_svm = svm.SVC()
# clf_grid = [clf_forest, clf2_boost, clf3_svm]

gc = GridSearchCV(estimator=clf_forest, param_grid=param_grid, cv=4, n_jobs=1, scoring='accuracy', verbose=2, refit=True)
gc.fit(X_train, y_train)

preds_test = gc.predict(X_test)
preds_train = gc.predict(X_train)
# print(preds_test)
# print(len(preds_test))
# preds_train = gc.predict(X_train)

# preds_proba_test = gc.predict_proba(X_test)
# print(preds_proba_test)
# print(len(preds_proba_test))
# preds_proba_train = gc.predict_proba(X_train)
# print(preds_proba_train)

#         test_logg.append(np.mean(y_test==preds_test))
#         train_logg.append(np.mean(y_train==preds_train))
print(np.mean(y_test==preds_test))
print(np.mean(y_train==preds_train))
print(preds_test)
print(preds_train)
print(y_train==preds_train)
# print(np.mean(y_train==preds_train))
print(gc.best_params_)


Fitting 4 folds for each of 9 candidates, totalling 36 fits
[CV] END ........................................max_depth=4; total time=   0.2s
[CV] END ........................................max_depth=4; total time=   0.2s
[CV] END ........................................max_depth=4; total time=   0.1s
[CV] END ........................................max_depth=4; total time=   0.2s
[CV] END ........................................max_depth=5; total time=   0.1s
[CV] END ........................................max_depth=5; total time=   0.2s
[CV] END ........................................max_depth=5; total time=   0.2s
[CV] END ........................................max_depth=5; total time=   0.1s
[CV] END ........................................max_depth=6; total time=   0.1s
[CV] END ........................................max_depth=6; total time=   0.2s
[CV] END ........................................max_depth=6; total time=   0.1s
[CV] END ........................................

In [41]:
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, preds_test)
print(clf_report)

              precision    recall  f1-score   support

           0       0.57      0.92      0.70       483
           1       0.26      0.04      0.07       348

    accuracy                           0.55       831
   macro avg       0.42      0.48      0.39       831
weighted avg       0.44      0.55      0.44       831



In [24]:
import sys
clf_report = classification_report(y_train, preds_train)
print(clf_report, file=sys.stdout)

              precision    recall  f1-score   support

           0       0.61      1.00      0.76       281
           1       1.00      0.17      0.30       219

    accuracy                           0.64       500
   macro avg       0.80      0.59      0.53       500
weighted avg       0.78      0.64      0.55       500



In [22]:
estimator = gc.best_estimator_
joblib.dump(estimator, "ML_model.joblib")

['ML_model.joblib']

# Результат: точность модели 0.73 на тестовых данных.

# Результат 2: точность на тестовых 0.92?!

## Вероятно, из-за небольшой выборки и перекоса к отрицательным результатам на почти всех компаниях

## Результат 2 сохраняется при перезапусках

# Результат 3 - из оценки тональности убрана тестовая выборка (все слова, которые не встречаличь ранее - теперь неизвестны). Итог: 0.92! Уровень сохранен

# Результат 4 - входные столбцы модели только из тренировочной выборки - как в работе - точность падает со временем (количество совпадающих в статьях слов уменьшается). Точность от 0.91 до 0.72 на сотнях статей (100 - 200)

# Результат 5 - берем больше тестовую выборку, проверяем что модель не выдает только 0. Точность 0.72.

# Итерация 6 - смешиваем обычный и взвешенный tf-idf

## Неудачный

In [39]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [40]:
news = pd.read_csv(f'news_links/final_v5.csv')

# print(news.columns)
    
news['message'] = news['title'] + ' ' + news['announce'] + ' ' + news['text']

news.drop(columns=['link', 'title', 'announce', 'text', 'title origin', 'announce origin', 'text origin'], inplace=True)
print(news.isna().sum())

date               0
ABRD close prev    0
ABRD close         0
AFLT close prev    0
AFLT close         0
                  ..
real_estate        0
consume_retail     0
tech_market        0
research           0
message            0
Length: 80, dtype: int64


In [41]:
print(news)

                      date  ABRD close prev  ABRD close  AFLT close prev  \
0      2017-01-03 22:22:00             99.5        99.5           151.68   
1      2017-01-03 22:47:02             99.5        99.5           151.68   
2      2017-01-03 23:29:32             99.5        99.5           151.68   
3      2017-01-04 06:36:23             99.5        99.5           151.68   
4      2017-01-04 10:40:20             99.5        99.5           151.68   
...                    ...              ...         ...              ...   
90209  2025-03-31 20:58:57            207.6       203.6            69.47   
90210  2025-03-31 21:19:34            207.6       203.6            69.47   
90211  2025-03-31 21:56:33            207.6       203.6            69.47   
90212  2025-03-31 22:02:42            207.6       203.6            69.47   
90213  2025-03-31 22:13:58            207.6       203.6            69.47   

       AFLT close  ALRS close prev  ALRS close  APTK close prev  APTK close  \
0       

In [42]:
companies = {
    'ABRD': 'Абрау-Дюрсо ПАО ао', 
    'AFLT': 'Аэрофлот-росс.авиалин(ПАО)ао', 
    'ALRS': 'АЛРОСА ПАО ао',
    'APTK': 'ПАО "Аптечная сеть 36,6" ао',
    'CHMF': 'Северсталь (ПАО)ао',
    'GAZP': '"Газпром" (ПАО) ао', 
    'GMKN': 'ГМК "Нор.Никель" ПАО ао', 
    'LKOH': 'НК ЛУКОЙЛ (ПАО) - ао',
    'LSRG': 'Группа ЛСР ПАО ао',
    'MGNT': '"Магнит" ПАО ао',
    'MGTS': '"ПАО "МГТС" ао',
    'MOEX': 'ПАО Московская Биржа', 
    'MVID': '"М.видео" ПАО ао',
    'NAUK': 'НПО Наука ао',
    'PIKK': 'ПИК СЗ (ПАО) ао', 
    'PLZL': 'Полюс ПАО ао',
    'RBCM': 'ГК РБК ПАО ао',
    'ROSN': 'ПАО НК Роснефть',
    'RTKM': 'Ростелеком (ПАО) ао.',  
    'SBER': 'Сбербанк России ПАО ао',
    'TATN': 'ПАО "Татнефть" ао',
    'VTBR': 'ао ПАО Банк ВТБ'
    }

In [46]:
df = pd.DataFrame()
for company in ["GAZP"]:
    comp = news[news[company] == True].copy()
    comp['close'] = comp[f'{company} close'] - comp[f'{company} close prev']
    comp = comp.loc[:, ['message', 'close', 'date']]
    df = pd.concat([df, comp], axis=0)
df.drop_duplicates(inplace=True)
df.sort_values('date', ignore_index=True, inplace=True)
      
            
print(df)

                                                message  close  \
0     иран допускать газпром лукойл участие тендер н...  -0.26   
1     нафтогаз называть способ сохранение украина тр...  -3.44   
2     газпром увеличивать заявка транзит газ украина...   0.10   
3     газпром устанавливать новый рекорд поставка га...   0.10   
4     газпром трижды начало год зафиксировать рекорд...   0.30   
...                                                 ...    ...   
5642  фондовый рынок арестовывать рост индекс мосбир...  -7.48   
5643  ariston задавать программа возвращение условие...  -7.48   
5644  путин разрешать иностранец платить газ 1 июль ...  -7.48   
5645  газпром существенно увеличивать поставка слова...  -2.44   
5646  газ потребитель германия 2021 год подорожать 8...  -2.44   

                     date  
0     2017-01-04 14:05:44  
1     2017-01-05 12:38:43  
2     2017-01-06 16:02:03  
3     2017-01-06 17:43:35  
4     2017-01-07 16:30:33  
...                   ...  
5642  2025-

In [47]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=0.05, max_df=0.9, ngram_range=(1,1))

train, test = df.iloc[:int(0.8*len(comp)), :], df.iloc[int(0.8*len(comp)):, :]
tfidf.fit(train['message'])
vocab = tfidf.vocabulary_.keys()


# features_train = tfidf.transform(train['message'])
# features_test = tfidf.transform(test['message'])


In [48]:
print(vocab)
# print(features_test[10])

dict_keys(['допускать', 'лукойл', 'участие', 'нефтегазовый', 'проект', 'no', 'announce', '29', 'крупный', 'международный', 'компания', 'сообщать', 'национальный', 'нефтяной', 'число', 'участник', 'входить', 'китай', 'также', 'российский', 'основа', 'отношение', 'становиться', 'контракт', 'который', 'совместный', 'предприятие', 'образ', 'работать', 'условие', 'кроме', 'добыча', 'пакет', 'весь', 'программа', 'включать', '23', 'газовый', 'месторождение', '18', 'экономический', 'сотрудничество', 'москва', 'читать', 'материал', 'россия', 'нафтогаз', 'называть', 'сохранение', 'украина', 'транзит', 'газ', 'оставаться', 'транзитный', 'страна', 'европейский', 'управление', 'свой', 'газотранспортный', 'система', 'заявлять', 'глава', 'андрей', 'интервью', 'украинский', 'канал', 'слово', 'готовый', 'часть', 'бизнес', '2019', 'год', 'истекать', 'срок', 'действие', 'господин', 'отмечать', 'случай', 'реализация', 'nord', 'stream', 'турецкий', 'поток', 'мочь', 'поставлять', 'упасть', '19', 'согласно',

In [49]:
ml_tf_df = pd.DataFrame(columns=['message'])

for ind, row in df.iterrows():
    s = []
    for word in vocab:
        if word in row['message']:
            s.append(word)
     
    ml_tf_df = pd.concat([ml_tf_df, pd.DataFrame([' '.join(s)])], ignore_index=True)
        
        
print(ml_tf_df)

     message                                                  0
0        NaN  допускать лукойл участие нефтегазовый проект n...
1        NaN  проект no announce компания сообщать российски...
2        NaN  no announce сообщать российский становиться ве...
3        NaN  no announce компания читать материал россия га...
4        NaN  проект no announce компания российский читать ...
...      ...                                                ...
5642     NaN  компания участник входить также российский осн...
5643     NaN  проект международный компания российский отнош...
5644     NaN  участие проект no announce сообщать также росс...
5645     NaN  no announce крупный компания сообщать национал...
5646     NaN  no announce крупный российский материал россия...

[5647 rows x 2 columns]


In [50]:
ml_tf_df.drop(columns=['message'], inplace=True)
ml_tf_df.rename(columns={0: 'message'}, inplace=True)
print(ml_tf_df)

                                                message
0     допускать лукойл участие нефтегазовый проект n...
1     проект no announce компания сообщать российски...
2     no announce сообщать российский становиться ве...
3     no announce компания читать материал россия га...
4     проект no announce компания российский читать ...
...                                                 ...
5642  компания участник входить также российский осн...
5643  проект международный компания российский отнош...
5644  участие проект no announce сообщать также росс...
5645  no announce крупный компания сообщать национал...
5646  no announce крупный российский материал россия...

[5647 rows x 1 columns]


In [51]:
ml_ready_df = pd.concat([ml_tf_df, df['close']], axis=1)
print(ml_ready_df)

                                                message  close
0     допускать лукойл участие нефтегазовый проект n...  -0.26
1     проект no announce компания сообщать российски...  -3.44
2     no announce сообщать российский становиться ве...   0.10
3     no announce компания читать материал россия га...   0.10
4     проект no announce компания российский читать ...   0.30
...                                                 ...    ...
5642  компания участник входить также российский осн...  -7.48
5643  проект международный компания российский отнош...  -7.48
5644  участие проект no announce сообщать также росс...  -7.48
5645  no announce крупный компания сообщать национал...  -2.44
5646  no announce крупный российский материал россия...  -2.44

[5647 rows x 2 columns]


In [52]:
ml_ready_df['class'] = ml_ready_df['close'] > 0
ml_ready_df.drop(columns=['close'], inplace=True)
ml_ready_df = ml_ready_df.replace({True: 1, False: 0})
ml_ready_df = ml_ready_df.fillna(0)
print(ml_ready_df)

                                                message  class
0     допускать лукойл участие нефтегазовый проект n...      0
1     проект no announce компания сообщать российски...      0
2     no announce сообщать российский становиться ве...      1
3     no announce компания читать материал россия га...      1
4     проект no announce компания российский читать ...      1
...                                                 ...    ...
5642  компания участник входить также российский осн...      0
5643  проект международный компания российский отнош...      0
5644  участие проект no announce сообщать также росс...      0
5645  no announce крупный компания сообщать национал...      0
5646  no announce крупный российский материал россия...      0

[5647 rows x 2 columns]


C:\Users\zolgu\AppData\Local\Temp\ipykernel_276\304057744.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ml_ready_df = ml_ready_df.replace({True: 1, False: 0})


In [53]:
print(ml_ready_df['class'].value_counts())

class
0    2880
1    2767
Name: count, dtype: int64


In [54]:
import ir

tf_weight_df = ml_ready_df.loc[:int(0.8*len(ml_ready_df))]
docs = []
for ind, row in tf_weight_df.iterrows():
    docs.append((row['class'], row['message']))

index = ir.SentimentIndex('delta', 'unigram')
index.get_class = lambda x: x[0]
index.get_text = lambda x: x[1]
index.build(docs)

x = []
y = []
for doc in docs:
    x.append(index.weight(index.features(doc)))
    y.append(doc[0])
   
# Влияние тестовых данных на алгоритм тональности
tf_weight_df_end = ml_ready_df.loc[len(tf_weight_df):]


docs2 = []
vals = []
for ind, row in tf_weight_df_end.iterrows():
    docs2.append((0, row['message']))
    vals.append(row['class'])
    
i = 0
for doc in docs2:
    x.append(index.weight(index.features(doc)))
    y.append(vals[i])
    i += 1

In [55]:
print(x[0])

{'допускать': -6.361302477572996, 'лукойл': -6.660575149839686, 'участие': -6.876264611890766, 'нефтегазовый': -6.678342114654332, 'проект': -8.116417072794205, 'no': -8.683893367307235, 'announce': -8.602820277383675, '29': -6.626717749249025, 'крупный': -7.791109510610028, 'международный': -6.8679744089702925, 'компания': -8.871365005136852, 'сообщать': -8.52615293278771, 'национальный': -6.450470422144176, 'нефтяной': -6.882437470997847, 'число': -7.543802867501509, 'участник': -6.970730078143525, 'входить': -7.04315991598834, 'китай': -6.703188113240863, 'также': -8.417372856134026, 'российский': -8.614864218589684, 'основа': -7.080867896690782, 'отношение': -7.3098814858247865, 'становиться': -8.086717920303906, 'контракт': -7.9603236291488395, 'который': -8.679822114864455, 'совместный': -6.44413125670044, 'предприятие': -6.827629234502853, 'образ': -7.354362330421477, 'работать': -7.144407180321139, 'условие': -7.7886260656250315, 'кроме': -7.233455418621439, 'добыча': -7.220373

In [56]:
ml_df = pd.DataFrame(x)
print(ml_df)

      допускать    лукойл   участие  нефтегазовый    проект        no  \
0     -6.361302 -6.660575 -6.876265     -6.678342 -8.116417 -8.683893   
1           NaN       NaN       NaN           NaN -8.116417 -8.683893   
2           NaN       NaN       NaN           NaN       NaN -8.683893   
3           NaN       NaN       NaN           NaN       NaN -8.683893   
4           NaN       NaN       NaN           NaN -8.116417 -8.683893   
...         ...       ...       ...           ...       ...       ...   
5642        NaN       NaN       NaN           NaN       NaN       NaN   
5643        NaN       NaN       NaN           NaN -8.116417       NaN   
5644        NaN       NaN -6.876265           NaN -8.116417 -8.683893   
5645        NaN       NaN       NaN           NaN       NaN -8.683893   
5646        NaN       NaN       NaN           NaN       NaN -8.683893   

      announce        29  крупный  международный  ...  исключать  подробность  \
0     -8.60282 -6.626718 -7.79111      -6.

In [57]:
ml_df = pd.concat([ml_df, ml_ready_df['class']], axis=1)
ml_df = ml_df.fillna(0)
print(ml_df)

      допускать    лукойл   участие  нефтегазовый    проект        no  \
0     -6.361302 -6.660575 -6.876265     -6.678342 -8.116417 -8.683893   
1      0.000000  0.000000  0.000000      0.000000 -8.116417 -8.683893   
2      0.000000  0.000000  0.000000      0.000000  0.000000 -8.683893   
3      0.000000  0.000000  0.000000      0.000000  0.000000 -8.683893   
4      0.000000  0.000000  0.000000      0.000000 -8.116417 -8.683893   
...         ...       ...       ...           ...       ...       ...   
5642   0.000000  0.000000  0.000000      0.000000  0.000000  0.000000   
5643   0.000000  0.000000  0.000000      0.000000 -8.116417  0.000000   
5644   0.000000  0.000000 -6.876265      0.000000 -8.116417 -8.683893   
5645   0.000000  0.000000  0.000000      0.000000  0.000000 -8.683893   
5646   0.000000  0.000000  0.000000      0.000000  0.000000 -8.683893   

      announce        29  крупный  международный  ...  подробность     2024  \
0     -8.60282 -6.626718 -7.79111      -6.86

In [58]:
train, test = ml_df.iloc[:int(0.8*len(ml_df)), :], ml_df.iloc[int(0.8*len(ml_df)):, :]
    

X_train = train.drop('class', axis=1)
X_test = test.drop('class', axis=1)

# print(X_test)

y_train = train['class'].values
y_test = test['class'].values

In [39]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import joblib
import numpy as np

In [60]:
param_grid = {'max_depth' : list(range(1,13))}


clf_forest = RandomForestClassifier(criterion='entropy', n_jobs=-1)

gc = GridSearchCV(estimator=clf_forest, param_grid=param_grid, cv=4, n_jobs=1, scoring='accuracy', verbose=2, refit=True)
gc.fit(X_train, y_train)

preds_test = gc.predict(X_test)
preds_train = gc.predict(X_train)


print(np.mean(y_test==preds_test))
# print(np.mean(y_train==preds_train))
# print(preds_test)
# print(preds_train)
# print(y_train==preds_train)
# print(np.mean(y_train==preds_train))
print(gc.best_params_)

Fitting 4 folds for each of 12 candidates, totalling 48 fits
[CV] END ........................................max_depth=1; total time=   0.2s
[CV] END ........................................max_depth=1; total time=   0.1s
[CV] END ........................................max_depth=1; total time=   0.1s
[CV] END ........................................max_depth=1; total time=   0.1s
[CV] END ........................................max_depth=2; total time=   0.1s
[CV] END ........................................max_depth=2; total time=   0.2s
[CV] END ........................................max_depth=2; total time=   0.2s
[CV] END ........................................max_depth=2; total time=   0.1s
[CV] END ........................................max_depth=3; total time=   0.2s
[CV] END ........................................max_depth=3; total time=   0.1s
[CV] END ........................................max_depth=3; total time=   0.2s
[CV] END .......................................

In [62]:
print(preds_test)
count = 0
for i in preds_test:
    count += int(i)
    
print(count)

[0 0 0 ... 0 1 0]
234


In [61]:
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, preds_test)
print(clf_report)

              precision    recall  f1-score   support

           0       0.54      0.80      0.65       612
           1       0.47      0.21      0.29       518

    accuracy                           0.53      1130
   macro avg       0.50      0.50      0.47      1130
weighted avg       0.51      0.53      0.48      1130



# Итерация 8?

In [1]:
import pandas as pd
import ir
from sklearn.ensemble import RandomForestClassifier
# from lightgbm import LGBMClassifier
# from sklearn import svm
from sklearn.model_selection import GridSearchCV
import os
import pandas as pd
import numpy as np
import joblib
from sklearn.metrics import classification_report


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "D:\PROgrams_for_teach\anaconda\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "D:\PROgrams_for_teach\anaconda\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "D:\PROgrams_for_teach\anaconda\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "D:\PROgrams_for_teach\anaconda\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
    app.start()
  File "D:\PROgrams_fo

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "D:\PROgrams_for_teach\anaconda\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "D:\PROgrams_for_teach\anaconda\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "D:\PROgrams_for_teach\anaconda\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "D:\PROgrams_for_teach\anaconda\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
    app.start()
  File "D:\PROgrams_fo

AttributeError: _ARRAY_API not found

In [2]:
companies = {
    'ABRD': 'Абрау-Дюрсо ПАО ао', 
    'AFLT': 'Аэрофлот-росс.авиалин(ПАО)ао', 
    'ALRS': 'АЛРОСА ПАО ао',
    'APTK': 'ПАО "Аптечная сеть 36,6" ао',
    'CHMF': 'Северсталь (ПАО)ао',
    'GAZP': '"Газпром" (ПАО) ао', 
    'GMKN': 'ГМК "Нор.Никель" ПАО ао', 
    'LKOH': 'НК ЛУКОЙЛ (ПАО) - ао',
    'LSRG': 'Группа ЛСР ПАО ао',
    'MGNT': '"Магнит" ПАО ао',
    'MGTS': '"ПАО "МГТС" ао',
    'MOEX': 'ПАО Московская Биржа', 
    'MVID': '"М.видео" ПАО ао',
    'NAUK': 'НПО Наука ао',
    'PIKK': 'ПИК СЗ (ПАО) ао', 
    'PLZL': 'Полюс ПАО ао',
    'RBCM': 'ГК РБК ПАО ао',
    'ROSN': 'ПАО НК Роснефть',
    'RTKM': 'Ростелеком (ПАО) ао.',  
    'SBER': 'Сбербанк России ПАО ао',
    'TATN': 'ПАО "Татнефть" ао',
    'VTBR': 'ао ПАО Банк ВТБ'
    }

In [3]:
news = pd.read_csv(f'news_links/final_v5.csv')

# news['message'] = news['title'] + ' ' + news['announce'] + ' ' + news['text']
    
news.drop(columns=['link', 'title', 'announce', 'text'], inplace=True)
print(news.isna().sum())

date               0
title origin       0
announce origin    0
text origin        0
ABRD close prev    0
                  ..
it                 0
real_estate        0
consume_retail     0
tech_market        0
research           0
Length: 82, dtype: int64


In [4]:
df = pd.DataFrame()
for company in ["GAZP"]:
    comp = news[news[company] == True].copy()
    comp['close'] = comp[f'{company} close'] - comp[f'{company} close prev']
    comp = comp.loc[:, ['title origin', 'announce origin', 'text origin', 'close', 'date',
        'ABRD', 'AFLT', 'ALRS', 'APTK', 'CHMF', 'GAZP',
        'GMKN', 'LKOH', 'LSRG', 'MGNT', 'MGTS', 'MOEX', 'MVID', 'NAUK', 'PIKK',
        'PLZL', 'RBCM', 'ROSN', 'RTKM', 'SBER', 'TATN', 'VTBR', 'finance',
        'gold', 'ferrous_metallurgy', 'oil_gas', 'non_ferrous_metallurgy',
        'electrical networks', 'telecom', 'it', 'real_estate', 'consume_retail',
        'tech_market', 'research']]
    df = pd.concat([df, comp], axis=0)
df.drop_duplicates(inplace=True)
df.sort_values('date', ignore_index=True, inplace=True)
            
            
print(df)

                                           title origin  \
0     иран допустил «газпром» и лукойл к участию в т...   
1     «нафтогаз» назвал способ сохранения украиной т...   
2     «газпром» увеличил заявку на транзит газа чере...   
3     «газпром» установил новый рекорд поставок газа...   
4     «газпром» трижды с начала года зафиксировал ре...   
...                                                 ...   
5640                    фондовому рынку арестовали рост   
5641               ariston задала программу возвращения   
5642  путин разрешил иностранцам платить за газ до 1...   
5643  «газпром» существенно увеличит поставки в слов...   
5644  газ для потребителей в германии с 2021 года по...   

                                        announce origin  \
0                                           no announce   
1                                           no announce   
2                                           no announce   
3                                           no announce

In [6]:
comp_test = df[df['GAZP'] == True]
print(len(comp_test))

5645


In [7]:
title_origin_df = comp_test.loc[0:len(comp_test) - 501, ['title origin', 'close']]
print(len(title_origin_df))
title_origin_end = comp_test.loc[len(title_origin_df):, ['title origin', 'close']]
print(len(title_origin_end))

5145
500


In [8]:
import ir
title_origin_df = comp_test.loc[0:len(comp_test) - 500, ['title origin', 'close']]
docs = []
for ind, row in title_origin_df.iterrows():
    docs.append((row['close'], row['title origin']))


index = ir.SentimentIndex('delta', 'bogram')
index.get_class = lambda x: x[0]
index.get_text = lambda x: x[1]
index.build(docs)

x = []
y = []
for doc in docs:
    x.append(index.weight(index.features(doc)))
    y.append(doc[0])
   
# Тест на влияние тестовых данных на алгоритм тональности
title_origin_end = comp_test.loc[len(title_origin_df):, ['title origin', 'close']]

docs2 = []
vals = []
for ind, row in title_origin_end.iterrows():
    docs2.append((0, row['title origin']))
    vals.append(row['close'])
    

i = 0
for doc in docs2:
    x.append(index.weight(index.features(doc)))
    y.append(vals[i])
    i += 1


In [9]:
print(len(comp_test) - 1000)
print(len(comp_test) - 500)

4645
5145


In [10]:
ml_ready_df = pd.DataFrame()

for row in x[len(comp_test) - 1000:len(comp_test) - 500]:
    ml_ready_df = pd.concat([ml_ready_df, pd.DataFrame([row])], ignore_index=True)
    
cols = ml_ready_df.columns.tolist()

for row in x[len(comp_test) - 500:]:
    ml_ready_df = pd.concat([ml_ready_df, pd.DataFrame([row])], ignore_index=True).loc[:, cols]


print(ml_ready_df)

         где     тонко      туда         и  где тонко  тонко туда    туда и  \
0   -1.94591 -1.098612 -1.098612 -6.658011  -1.098612   -1.098612 -1.098612   
1        NaN       NaN       NaN       NaN        NaN         NaN       NaN   
2        NaN       NaN       NaN       NaN        NaN         NaN       NaN   
3        NaN       NaN       NaN       NaN        NaN         NaN       NaN   
4        NaN       NaN       NaN       NaN        NaN         NaN       NaN   
..       ...       ...       ...       ...        ...         ...       ...   
995      NaN       NaN       NaN       NaN        NaN         NaN       NaN   
996      NaN       NaN       NaN       NaN        NaN         NaN       NaN   
997      NaN       NaN       NaN       NaN        NaN         NaN       NaN   
998      NaN       NaN       NaN       NaN        NaN         NaN       NaN   
999      NaN       NaN       NaN       NaN        NaN         NaN       NaN   

     владимир     путин  поздравил  ...  бензина  п

In [ ]:
print(len(x))
print(len(comp_test) - 500)
print(len(title_origin_end))

In [22]:
# print("Размер датасета: ", len(x))
# print(len(y))
r = x[-2]
print("Значения взвешенного TF-IDF: ", r)
# for key, value in r.items():  
#     r[key] = round(r[key],2)
#     print(key, r[key], ' ')
#     if c %  == 
print("Изменение цены: ", round(y[-2], 2))

Значения взвешенного TF-IDF:  {'газпром': -7.64, 'существенно': 0, 'увеличит': -2.94, 'поставки': -5.61, 'в': -8.0, 'словакию': -1.61, 'через': -5.8, 'турецкий': -3.89, 'газпром существенно': 0, 'существенно увеличит': 0, 'увеличит поставки': -1.95, 'поставки в': -2.94, 'в словакию': -1.1, 'словакию через': -1.1, 'через турецкий': -1.61}
Изменение цены:  -2.44


In [18]:
r = x[-2]
# print(type(r))
for key, value in r.items():  
    r[key] = round(r[key],2)
print(r)

{'газпром': -7.64, 'существенно': 0, 'увеличит': -2.94, 'поставки': -5.61, 'в': -8.0, 'словакию': -1.61, 'через': -5.8, 'турецкий': -3.89, 'газпром существенно': 0, 'существенно увеличит': 0, 'увеличит поставки': -1.95, 'поставки в': -2.94, 'в словакию': -1.1, 'словакию через': -1.1, 'через турецкий': -1.61}


In [34]:
sc = pd.Series(y[len(comp_test) - 1000:])

ml_ready_df = pd.concat([ml_ready_df, sc], axis=1)
ml_ready_df.rename(columns={0: 'close'}, inplace=True)
print(ml_ready_df)

         где     тонко      туда         и  где тонко  тонко туда    туда и  \
0   -1.94591 -1.098612 -1.098612 -6.658011  -1.098612   -1.098612 -1.098612   
1        NaN       NaN       NaN       NaN        NaN         NaN       NaN   
2        NaN       NaN       NaN       NaN        NaN         NaN       NaN   
3        NaN       NaN       NaN       NaN        NaN         NaN       NaN   
4        NaN       NaN       NaN       NaN        NaN         NaN       NaN   
..       ...       ...       ...       ...        ...         ...       ...   
995      NaN       NaN       NaN       NaN        NaN         NaN       NaN   
996      NaN       NaN       NaN       NaN        NaN         NaN       NaN   
997      NaN       NaN       NaN       NaN        NaN         NaN       NaN   
998      NaN       NaN       NaN       NaN        NaN         NaN       NaN   
999      NaN       NaN       NaN       NaN        NaN         NaN       NaN   

     владимир     путин  поздравил  ...  полгода  р

In [35]:
test_df = ml_ready_df.copy()
test_df['class'] = test_df['close'] > 0
test_df.drop(columns=['close'], inplace=True)
test_df = test_df.replace({True: 1, False: 0})
test_df = test_df.fillna(0)
print(test_df)

         где     тонко      туда         и  где тонко  тонко туда    туда и  \
0   -1.94591 -1.098612 -1.098612 -6.658011  -1.098612   -1.098612 -1.098612   
1    0.00000  0.000000  0.000000  0.000000   0.000000    0.000000  0.000000   
2    0.00000  0.000000  0.000000  0.000000   0.000000    0.000000  0.000000   
3    0.00000  0.000000  0.000000  0.000000   0.000000    0.000000  0.000000   
4    0.00000  0.000000  0.000000  0.000000   0.000000    0.000000  0.000000   
..       ...       ...       ...       ...        ...         ...       ...   
995  0.00000  0.000000  0.000000  0.000000   0.000000    0.000000  0.000000   
996  0.00000  0.000000  0.000000  0.000000   0.000000    0.000000  0.000000   
997  0.00000  0.000000  0.000000  0.000000   0.000000    0.000000  0.000000   
998  0.00000  0.000000  0.000000  0.000000   0.000000    0.000000  0.000000   
999  0.00000  0.000000  0.000000  0.000000   0.000000    0.000000  0.000000   

     владимир     путин  поздравил  ...  полгода  р

C:\Users\zolgu\AppData\Local\Temp\ipykernel_276\1256897716.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_df = test_df.replace({True: 1, False: 0})


In [36]:
train, test = test_df.iloc[:int(0.5*len(test_df)), :], test_df.iloc[int(0.5*len(test_df)):, :]
    

X_train = train.drop('class', axis=1)
X_test = test.drop('class', axis=1)

print(X_test)

y_train = train['class'].values
y_test = test['class'].values

print(y_test)
print(len(y_test))

     где  тонко  туда    и  где тонко  тонко туда  туда и  владимир     путин  \
500  0.0    0.0   0.0  0.0        0.0         0.0     0.0       0.0  0.000000   
501  0.0    0.0   0.0  0.0        0.0         0.0     0.0       0.0  0.000000   
502  0.0    0.0   0.0  0.0        0.0         0.0     0.0       0.0  0.000000   
503  0.0    0.0   0.0  0.0        0.0         0.0     0.0       0.0  0.000000   
504  0.0    0.0   0.0  0.0        0.0         0.0     0.0       0.0  0.000000   
..   ...    ...   ...  ...        ...         ...     ...       ...       ...   
995  0.0    0.0   0.0  0.0        0.0         0.0     0.0       0.0  0.000000   
996  0.0    0.0   0.0  0.0        0.0         0.0     0.0       0.0  0.000000   
997  0.0    0.0   0.0  0.0        0.0         0.0     0.0       0.0 -5.241747   
998  0.0    0.0   0.0  0.0        0.0         0.0     0.0       0.0  0.000000   
999  0.0    0.0   0.0  0.0        0.0         0.0     0.0       0.0  0.000000   

     поздравил  ...  бензин

In [37]:
param_grid = {'max_depth' : list(range(1,13))}


clf_forest = RandomForestClassifier(criterion='entropy', n_jobs=-1)
# clf2_boost = LGBMClassifier(random_state=123, n_jobs=-1)
# clf3_svm = svm.SVC()
# clf_grid = [clf_forest, clf2_boost, clf3_svm]

gc = GridSearchCV(estimator=clf_forest, param_grid=param_grid, cv=4, n_jobs=1, scoring='accuracy', verbose=2, refit=True)
gc.fit(X_train, y_train)

preds_test = gc.predict(X_test)
preds_train = gc.predict(X_train)
# print(preds_test)
# print(len(preds_test))
# preds_train = gc.predict(X_train)

# preds_proba_test = gc.predict_proba(X_test)
# print(preds_proba_test)
# print(len(preds_proba_test))
# preds_proba_train = gc.predict_proba(X_train)
# print(preds_proba_train)

#         test_logg.append(np.mean(y_test==preds_test))
#         train_logg.append(np.mean(y_train==preds_train))
print(np.mean(y_test==preds_test))
print(np.mean(y_train==preds_train))
print(preds_test)
print(preds_train)
print(y_train==preds_train)
# print(np.mean(y_train==preds_train))
print(gc.best_params_)

Fitting 4 folds for each of 12 candidates, totalling 48 fits
[CV] END ........................................max_depth=1; total time=   0.3s
[CV] END ........................................max_depth=1; total time=   0.1s
[CV] END ........................................max_depth=1; total time=   0.1s
[CV] END ........................................max_depth=1; total time=   0.1s
[CV] END ........................................max_depth=2; total time=   0.1s
[CV] END ........................................max_depth=2; total time=   0.1s
[CV] END ........................................max_depth=2; total time=   0.2s
[CV] END ........................................max_depth=2; total time=   0.1s
[CV] END ........................................max_depth=3; total time=   0.1s
[CV] END ........................................max_depth=3; total time=   0.1s
[CV] END ........................................max_depth=3; total time=   0.1s
[CV] END .......................................

In [38]:
clf_report = classification_report(y_test, preds_test)
print(clf_report)

              precision    recall  f1-score   support

           0       0.52      0.94      0.67       264
           1       0.26      0.03      0.05       236

    accuracy                           0.51       500
   macro avg       0.39      0.48      0.36       500
weighted avg       0.40      0.51      0.37       500

